In [14]:
import sys
import numpy as np
import matplotlib.pyplot as plt
#import uproot
import pandas as pd
import scipy.optimize as opt
import math
import statistics as st
import os
import PyQt5
from numba import njit
import time
import numba
from numba.typed import List
from numba import uint16, intc, uint32
from array import array

print(os.getcwd())
os.chdir('F:/LANL/')
datadir = 'sample_data/'
folder = 'La_sample/'
run_num = "11143" 
print(os.getcwd() + folder)
statefileloc = 'D:\LANL\SF_Norm_files\TR_R_expected_avgs_stds_afterclip.csv'
SFNormFile = 'SF_Norm_files/'+folder+run_num
processedfolder = 'processed_data/'
AddedPulseSavename = processedfolder+run_num+'_pulsesadded'
AsymSavename = processedfolder+run_num+'_asym'

%matplotlib qt

start = time.time()
fullstart = time.time()

#chan_enab = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]
chan_enab = np.array([0,1,2,3,4,5,18,19,20,21,22,23,24])
#chan_enab = np.array([1,19,24])


#@jit(nopython = True)
# read_data = np.array([])
# fileLength = np.array([])
read_data = []
fileLength = []


def open_file():
    for el in chan_enab:
        f = open(datadir + folder + 'run' + run_num + "_ch" + str(el) + ".bin", 'rb')
        read_data.append(np.fromfile(file=f, dtype=np.uint16))
        #chdat = np.fromfile(file=f, dtype=np.uint16)
        #print((len(chdat)))
        #read_data = np.vstack(read_data,chdat)
        #print(len(read_data))
        f.close()
        #print((read_data))
        fileLength.append(len(read_data[-1]))
        #np.append(fileLength,len(read_data[-1]))
        #read_data = np.asarray(read_data)
    return read_data, fileLength


#np.asarray(fileLength)
open_file()
#print(read_data)
#print(type(fileLength))
#print(type(read_data))
#np.asarray(read_data)
fileLength = np.asarray(fileLength)
read_data = np.asarray(read_data) ## in detector's case, all are the same size samples, so can do read_data as np array

if chan_enab[-1] != 24:
    raise Exception('last channel is not 6Li detector')

end = time.time()
print('file open time: ' + str(end-start))            
#print(read_data)

print(AddedPulseSavename)
print(AsymSavename)

F:\LANL\analysis
F:\LANLLa_sample/
file open time: 2.6782453060150146
processed_data/11143_pulsesadded
processed_data/11143_asym


Store the big header for each channel in arrays

In [16]:
BoardID = []
recordLength = []
numSamples = []
eventCounter = []
decFactor = []
chanDec = []
postTrig = []
groupStart = []
groupEnd = []
timestamp= []
sizeFirstEvent = []
TTT = []

targetDict = {0: "La", 1: "Tb2O3", 2: "Yb2O3", 3: "Sm2O3", 4: "Er2O3", 5: "Ho2O3", 6: "other"}
foilDict = {0: "TBD", 1: "TBD", 2: "TBD", 3: "TBD", 4: "TBD", 5: "TBD", 6: "other"}

target=(read_data[0][5]&0x00F0)>>4
foil=read_data[0][5]&0x000F
targetFlag = read_data[0][5]>>8&1
foilFlag = read_data[0][5]>>9&1
spinFiltFlag = read_data[0][5]>>10&1
spinFlipFlag = read_data[0][5]>>11&1
shutterFlag = read_data[0][5]>>12&1
facilityTrigFlag = read_data[0][5]>>13&1

if targetFlag:
    target=targetDict[(read_data[0][5]&0x00F0)>>4]
    
else:
    target = "empty"
    
    
if foilFlag:
    foil=foilDict[read_data[0][5]&0x000F]
else:
    foil = "empty"

for i in range(0,len(chan_enab)):
    BoardID.append(read_data[i][9]>>8)
    recordLength.append(((read_data[i][9]&0x00FF)<<16)+read_data[i][8])
    numSamples.append(((read_data[i][11]&0x00FF)<<16)+read_data[i][10])
    eventCounter.append(read_data[i][6]+(read_data[i][7]<<16))
    BoardID.append(read_data[i][9]>>8)  
    decFactor.append(read_data[i][11]>>8)
    chanDec.append(read_data[i][13]>>8)
    postTrig.append(read_data[i][15]>>8)
    groupStart.append(((read_data[i][13]&0x00FF)<<16)+read_data[i][12])
    groupEnd.append(((read_data[i][15]&0x00FF)<<16)+read_data[i][14])
    
    timestamp.append(read_data[i][16]+(read_data[i][17]<<16)+(read_data[i][18]<<32)+(read_data[i][19]<<40))  
    sizeFirstEvent.append(read_data[i][0]+(read_data[i][1]<<16))
    TTT.append(read_data[i][2]+(read_data[i][3]<<16)+(read_data[i][4]<<32))
    
    print("For channel " + str(chan_enab[i]) + ", BoardID is " + str(BoardID[i])
          + "; record length is " + str(recordLength[i]) + "; num Samples is " 
          + str(numSamples[i]) + "; event counter is " + str(eventCounter[i]) + "; dec factor is " + str(decFactor[i]) + "; chan dec is " 
          + str(chanDec[i]) + "; postTrig is " + str(postTrig[i]) + "; group start is " + str(groupStart[i]) + "; group end is " + str(groupEnd[i])
          + "; epoch time is " + str(timestamp[i]) +  "; first event size is " + str(sizeFirstEvent[i]) + "; and ETTT is " + str(TTT[i]) + "\n")

BoardID = np.asarray(BoardID) 
recordLength = np.asarray(recordLength)
numSamples = np.asarray(numSamples)
eventCounter = np.asarray(eventCounter)
decFactor = np.asarray(decFactor)
chanDec = np.asarray(chanDec)
postTrig = np.asarray(postTrig)
groupStart = np.asarray(groupStart)
groupEnd = np.asarray(groupEnd)
timestamp = np.asarray(timestamp)
sizeFirstEvent = np.asarray(sizeFirstEvent)
TTT = np.asarray(TTT)

print("Target is " + target)
print("Foil is " + foil)
print("Shutter is open: " + str(bool(shutterFlag)))
print("Facility t0 is on: " + str(bool(facilityTrigFlag)))
print("Spin flipper is on: " + str(bool(spinFlipFlag)))
print("Spin filter is on: " + str(bool(spinFiltFlag)))
print("Target is present: " + str(bool(targetFlag)))
print("Foil is present: " + str(bool(foilFlag)))


For channel 0, BoardID is 31; record length is 45000; num Samples is 9000; event counter is 20000; dec factor is 32; chan dec is 1; postTrig is 80; group start is 7000; group end is 16000; epoch time is 1706060412; first event size is 18040; and ETTT is 561986561

For channel 1, BoardID is 31; record length is 45000; num Samples is 9000; event counter is 20000; dec factor is 32; chan dec is 1; postTrig is 80; group start is 7000; group end is 16000; epoch time is 1706060412; first event size is 18040; and ETTT is 561986561

For channel 2, BoardID is 31; record length is 45000; num Samples is 9000; event counter is 20000; dec factor is 32; chan dec is 1; postTrig is 80; group start is 7000; group end is 16000; epoch time is 1706060412; first event size is 18040; and ETTT is 561986561

For channel 3, BoardID is 31; record length is 45000; num Samples is 9000; event counter is 20000; dec factor is 32; chan dec is 1; postTrig is 80; group start is 7000; group end is 16000; epoch time is 17

Determine the time axis for each channel

In [18]:
preTime = []
startTime = []
endTime = []
resolution = []
xs = [] 

for i in range(0,len(chan_enab)):
    preTime.append((100-postTrig[i])*recordLength[i]/100)
    startTime.append((-1*preTime[i]*16*decFactor[i] + groupStart[i]*16*decFactor[i]))
    endTime.append((-1*preTime[i]*16*decFactor[i] + groupEnd[i]*16*decFactor[i]))
    resolution.append(16*chanDec[i]*decFactor[i])
    print("Pretime for channel", chan_enab[i],"is " + str(preTime[i]) + "; start time is " + str(startTime[i]) + "; end time is " + str(endTime[i]) 
          + "; resolution is " + str(resolution[i]) + "ns")
    xs.append(np.arange(startTime[i],(numSamples[i])*resolution[i]+startTime[i], resolution[i]))

#np.asarray(preTime)
#np.asarray(startTime)
#np.asarray(endTime)
#np.asarray(resolution)
xs = np.asarray(xs) ## can convert xs to np array here because all detectors same numsamples
    
#print(len(xs[2]))
#print(preTime[0]/512)

Pretime for channel 0 is 9000.0; start time is -1024000.0; end time is 3584000.0; resolution is 512ns
Pretime for channel 1 is 9000.0; start time is -1024000.0; end time is 3584000.0; resolution is 512ns
Pretime for channel 2 is 9000.0; start time is -1024000.0; end time is 3584000.0; resolution is 512ns
Pretime for channel 3 is 9000.0; start time is -1024000.0; end time is 3584000.0; resolution is 512ns
Pretime for channel 4 is 9000.0; start time is -1024000.0; end time is 3584000.0; resolution is 512ns
Pretime for channel 5 is 9000.0; start time is -1024000.0; end time is 3584000.0; resolution is 512ns
Pretime for channel 18 is 9000.0; start time is -1024000.0; end time is 3584000.0; resolution is 512ns
Pretime for channel 19 is 9000.0; start time is -1024000.0; end time is 3584000.0; resolution is 512ns
Pretime for channel 20 is 9000.0; start time is -1024000.0; end time is 3584000.0; resolution is 512ns
Pretime for channel 21 is 9000.0; start time is -1024000.0; end time is 3584000

In [19]:
start=time.time()

@njit
def dataread(data, channels, fileLen, numSamps):
    #ys_arr = []
    numRuns = int((fileLen[0]-20-numSamps[0])/(numSamps[0]+6)+1)
    print(numRuns)
    ys_arr = np.zeros((len(channels), numRuns,numSamps[0]), dtype=np.uint16)
    ETTT_arr = np.zeros((len(channels), numRuns), dtype=np.intc)
    eventcount_arr = np.zeros((len(channels), numRuns), dtype=np.intc)
    for i in range(0,len(channels)):
        eventCount = 0
        byteCounter = 0
            #byte counter is really 2bytecounter, lol
        while byteCounter < fileLen[i]:
            if byteCounter == 0:
                ETTT_arr[i]=TTT[i]
                #ETTT_arr[i].append(TTT[i])
                eventcount_arr[i]=(eventCounter[i])
                byteCounter = 20
            else:
                ETTT_arr[i]=(data[i][byteCounter]+(data[i][byteCounter+1]<<16)+(data[i][byteCounter+2]<<32))
                eventcount_arr[i]=(data[i][byteCounter+4]+(data[i][byteCounter+5]<<16))
                byteCounter += 6
            for j in range(0, numSamps[i]):
                #if j == 0:
                    #ys_arr[i].append([])
                #print(byteCounter)
                ys_arr[i][eventCount][j]=data[i][byteCounter]
                byteCounter += 1
            eventCount += 1
    return ys_arr, ETTT_arr, eventcount_arr

#start=time.time()
#ys_arrHe, ETTT_arrHe, eventcount_arrHe  = dataread(read_data, [25], fileLength, numSamples) ##hardcoded channel 25 for He
ys_arr, ETTT_arr, eventcount_arr  = dataread(read_data, chan_enab, fileLength, numSamples) ##hardcoded channels for coils
#ys_arrtest, ETTT_arr, eventcount_arr  = dataread(read_data[1:], [12,24], fileLength[1:], numSamples[1:]) ##testing only!

end = time.time()
print('dataread from binary time: ' + str(end-start))            

#print(len(eventcount_arr))
# print('He:')
# print(len(ys_arrHe))
# print(len(ys_arrHe[0]))
#print(len(ys_arr[0]))
#print(len(ys_arr[0][0]))


5000
dataread from binary time: 3.5651073455810547


In [23]:
timeDif=[]
for i in range(0,len(chan_enab)):
    timeDif.append([])
    for j in range(len(ETTT_arr[i])-1):
        timeDif[i].append((ETTT_arr[i][j+1]-ETTT_arr[i][j])*8)
    print("Min time difference for channel", chan_enab[i], "is", min(timeDif[i]), "ns")
    print("Max time difference for channel", chan_enab[i], "is", max(timeDif[i]), "ns \n")
#print(timeDif)

Min time difference for channel 0 is 0 ns
Max time difference for channel 0 is 0 ns 

Min time difference for channel 1 is 0 ns
Max time difference for channel 1 is 0 ns 

Min time difference for channel 2 is 0 ns
Max time difference for channel 2 is 0 ns 

Min time difference for channel 3 is 0 ns
Max time difference for channel 3 is 0 ns 

Min time difference for channel 4 is 0 ns
Max time difference for channel 4 is 0 ns 

Min time difference for channel 5 is 0 ns
Max time difference for channel 5 is 0 ns 

Min time difference for channel 18 is 0 ns
Max time difference for channel 18 is 0 ns 

Min time difference for channel 19 is 0 ns
Max time difference for channel 19 is 0 ns 

Min time difference for channel 20 is 0 ns
Max time difference for channel 20 is 0 ns 

Min time difference for channel 21 is 0 ns
Max time difference for channel 21 is 0 ns 

Min time difference for channel 22 is 0 ns
Max time difference for channel 22 is 0 ns 

Min time difference for channel 23 is 0 ns
M

In [24]:
## basesub and plotting ##
baseL = 0
baseR = int(((preTime[0]-groupStart[0])*0.70)/chanDec[0])
numRuns = int((fileLength[0]-20-numSamples[0])/(numSamples[0]+6)+1)
    
ys_basesub = [] 
legend =  ['NaI', 'R']

start = time.time()

s = 20 ## pulse to look at
t=s+1

## dont know why this is so slow ##
def plotter(ys, xs, baseR, numpoints):
    tempys_basesub = np.zeros((len(ys), numRuns,numpoints[0]), dtype=float)
    for i in range((len(ys))):
        #tempys_basesub = []
        #tempys_basesub = np.zeros((len(ys), numRuns,numpoints[0]), dtype=float)
        #tempsums =[]
        for pulse in range((len(eventcount_arr[0]))): ## all have 5000 pulses
            tempys_basesub[i][pulse]=np.subtract(ys[i][pulse], np.mean(ys[i][pulse][baseL:baseR]))
        for j in range(s, t): ## plot only interested pulses
            #ys_basesub.append(ys_arr[i][j] - np.mean(ys_arr[i][j][200:6000]))
            #print(sum(ys_basesub[i][j])) 
            plt.plot(xs[i], tempys_basesub[i][j]) #label=legend[i]) #+str(sums[1][j])) ## sums[j] will not work for more than just TR   
            plt.axvline(xs[0][baseL], ls = '--')
            plt.axvline(xs[0][baseR], ls = '--')
            #plt.axvline(xs[0][int(((preTime[0]-groupStart[0])*0.70)/chanDec[0])], ls = '--', c ='m')
            plt.axvline(xs[0][baseR+5], ls = '--', c ='r') ## BaseR+5 line marks the beginning of the integral, until the end of samples.
#             plt.title('SF state transition' + transitions[p]) 
#             plt.xlabel("time from trigger (ns)")
#             plt.ylabel("ADC")
            plt.legend()
            
#plotter(ys_arr[9:], xs[9:], baseR, numSamples) ##plot coils

ys_basesub = np.zeros((len(ys_arr), numRuns,numSamples[0]), dtype=np.float64)

@njit ## jit is faster for large # channels, slower for small # channels
def basesub(ys, baseRight, numpoints): 
    tempys_basesub = np.zeros((numRuns,numpoints[0]), dtype=np.float64)
#     print(len(tempys_basesub))
#     print(len(tempys_basesub[0]))
#     tempsums = np.zeros((len(ys), numRuns), dtype=np.float64)
    for pulse in range((len(eventcount_arr[0]))): ## all have 5000 pulses
        tempys_basesub[pulse]=np.subtract(ys[pulse], np.mean(ys[pulse][baseL:baseRight]))
#             tempsums[i][pulse] = np.sum(tempys_basesub[i][pulse][baseR+5:-1])
    return tempys_basesub ##, tempsums

## got rid of sums here, should be done after aligning and cutting
## got rid of xs in basesub, don't think we need them as an input 06.10.24
        
#ys_basesub, sums = basesub_sum(ys_arrtest, xs[1:3], baseRCoil, numSamples[1:3]) ##testing only!!
for i in range(len(ys_basesub)):
    ys_basesub[i] = basesub(ys_arr[i], baseR, numSamples)

ys_basesub[-1] = ys_basesub[-1]*-1 ## invert 6Li to positive signal. Comment out if not using

end = time.time()
print('plotting and/or base subtraction time: ' + str(end-start))            
#print(len(sums))
#print(sumsHe[0][20:30])
#print(sums[1][20:30])

plotting and/or base subtraction time: 8.806713104248047


## Load SF Sorting and norm ##

In [27]:
## Load in SF and He normalization information 
df_SF = pd.read_hdf(SFNormFile + '.h5', key='df_0')
df_HE = pd.read_hdf(SFNormFile + '.h5', key='df_1')

# print(df_SF2)
# print(df_HE2)

SF_Sort_arr = df_SF[['nicknames', 'transition_locations']].to_numpy().T
He_Norm_arr = df_HE[['pulse', 'norms']].to_numpy().T

print((SF_Sort_arr))
print(len(SF_Sort_arr[1]))

# print(len(SF_Norm_arr[0]))
# print(SF_Norm_arr[1][0])
# print(He_Norm_arr[1][0])

NormFactor = 100000  ## He integrals are huge, this normalizes all of those by a constant value for ease of use
HeNorms= (He_Norm_arr[1])/NormFactor

print(HeNorms)
print(len(HeNorms))

[[0 1 2 3 4 5 6 7 0 1 2 3 4 5 6 7 0 1 2 3 4 5 6 7 0 1 2 3 4 5 6 7 0 1 2 3
  4 5 6 7 0 1 2 3 4 5 6 7 0 1 2 3 4 5 6 7 0 1 2 3 4 5 6 7 0 1 2 3 4 5 6 7
  0 1 2 3 4 5 6 7 0 1 2 3 4 5 6 7 0 1 2 3 4 5 6 7 0 1 2 3 4 5 6 7 0 1 2 3
  4 5 6]
 [15 60 105 150 195 240 285 330 375 420 465 510 555 600 645 690 735 780
  825 870 915 960 1005 1050 1095 1140 1185 1230 1275 1320 1365 1410 1455
  1500 1545 1590 1635 1680 1725 1770 1815 1860 1905 1950 1995 2040 2085
  2130 2175 2220 2265 2310 2355 2400 2445 2490 2535 2580 2625 2670 2715
  2760 2805 2850 2895 2940 2985 3030 3075 3120 3165 3210 3255 3300 3345
  3390 3435 3480 3525 3570 3615 3660 3705 3750 3795 3840 3885 3930 3975
  4020 4065 4110 4155 4200 4245 4290 4335 4380 4425 4470 4515 4560 4605
  4650 4695 4740 4785 4830 4875 4920 4965]]
111
[65.23310998 65.68454511 65.77308854 ... 64.68414914 63.4634804
 63.79080512]
5000


In [83]:
# for ind in df.index:
#     print('transition: '+ str(df['nicknames'][ind]) + ' location: ' + str(df['transition_locations'][ind]))
#print(df['transition_locations'],df['nicknames'] )

counter = 0
sequence = [[],[],[]]
seq = 0
smallerseq = []
smallerstateis = []

for i in range(len(SF_Sort_arr[1])-(np.mod((len(SF_Sort_arr[1])), 8))):  ##111 mod 8 = 7, so essentially 111-7 = 104
    counter = counter+1
    #print( i )
    #print(counter)
    if counter < 8:
#         smallersum.append(np.sum(sums_normed[0][(SF_Sort_arr[1][i])+5:(SF_Sort_arr[1][i+1])])) ## minus first 5 to get rid of transitioning pulses
        smallerstateis.append([(SF_Sort_arr[1][i])+5,(SF_Sort_arr[1][i+1])])
        smallerseq.append(SF_Sort_arr[0][i+1])
        # print(smallerseq)
        #print(smallerseq)
    elif counter == 8:
        seq = seq+1
        # print(smallerseq)
        smallerstateis.append([(SF_Sort_arr[1][i])+5,(SF_Sort_arr[1][i+1])])
        smallerseq.append(SF_Sort_arr[0][i+1])
        sequence[0].append(seq)
        sequence[1].append(smallerseq)   
        sequence[2].append(smallerstateis)
        smallerseq = []
        smallerstateis = []
#         smallersum = []
        counter  = 0
        #print(test_arr[1][i+1])
        
# print(counter)
# print(len(testsumarr))
#print(len(testsumarr))
# print(len(sequence))
# print(len(sequence[0]))
# print(len(sequence[1]))

# print(sequence[0])
# print(sequence[1][0])

# print(len(sequence))
# print(len(sequence[0]))
# print(len(sequence[1]))
# print(len(sequence[2]))
# print(len(sequence[1][0]))
# print(len(sequence[2][0]))

# # print((testsumarr))
# print(np.sum(testsums[0][35:75]))
# print(len(np.ndarray.flatten(np.asarray(sequence[1]))))
print(counter)

# leftovers = [[sequence[0][-1]+1],[],[]]

# for i in range((len(SF_Sort_arr[1])-(np.mod((len(SF_Sort_arr[1])), 8))), len(SF_Sort_arr[1])-1):
#     counter = counter+1
#     #print(counter)
#     if counter < 8:
#         print(counter)
#         leftovers[1].append(SF_Sort_arr[0][i+1])
#         leftovers[2].append([(SF_Sort_arr[1][i])+5,(SF_Sort_arr[1][i+1])])
# #         leftovers[2].append(np.sum(sums_normed[0][(SF_Sort_arr[1][i])+5:(SF_Sort_arr[1][i+1])])) ## minus first 5 to get rid of transitioning pulses
#     elif counter == 8: ## not really needed here, maybe delete
#         print(counter)
#         seq = seq+1
#         leftoversum.append(np.sum(sums_normed[0][(SF_Sort_arr[1][i])+5:(SF_Sort_arr[1][i+1])]))
#         smallerseq.append(SF_Sort_arr[0][i+1])
#         sequence[0].append(seq)
#         sequence[1].append(smallerseq)       
#         smallerseq = []
#         counter  = 0
#         #print(test_arr[1][i+1])

# try:
def find_leftover(SFsort_info):
    left = [[sequence[0][-1]+1],[],[]]
    counter = 0
    for i in range((len(SFsort_info[1])-(np.mod((len(SFsort_info[1])), 8))), len(SFsort_info[1])-1):
        counter = counter+1
        if counter < 8:
            left[1].append(SFsort_info[0][i+1])
            left[2].append([(SFsort_info[1][i])+5,(SFsort_info[1][i+1])])
    return left
# except Exception as e:
#     logger.error(run_num + ' failed during leftovers sequencing')
#     logger.exception(e)
leftovers = find_leftover(SF_Sort_arr)

print(leftovers)

print((sequence[2][0][0]))
# for p in range((sequence[2][0][0][0]),(sequence[2][0][0][1])):
#     print(p)

0
[[14], [1, 2, 3, 4, 5, 6], [[4700, 4740], [4745, 4785], [4790, 4830], [4835, 4875], [4880, 4920], [4925, 4965]]]
[20, 60]


In [91]:
# In[20]:

# for ind in df.index:
#     print('transition: '+ str(df['nicknames'][ind]) + ' location: ' + str(df['transition_locations'][ind]))
#print(df['transition_locations'],df['nicknames'] )

counter = 0
sequence = ([[],[],[]])
seq = 0
smallerseq = []
smallerstateis = []

# try:
for i in range(len(SF_Sort_arr[1])-(np.mod((len(SF_Sort_arr[1])), 8))):  ##111 mod 8 = 7, so essentially 111-7 = 104
    # print(i)
    counter = counter+1
    if counter < 8:
        if (SF_Sort_arr[1][i]) == 0:
            smallerstateis.append([(SF_Sort_arr[1][i])+5,(SF_Sort_arr[1][i+1])])
            smallerseq.append(SF_Sort_arr[0][i+1])
            seq = seq+1
            continue
        smallerstateis.append([(SF_Sort_arr[1][i])+5,(SF_Sort_arr[1][i+1])])
        smallerseq.append(SF_Sort_arr[0][i+1])
        # print(SF_Sort_arr[0][i+1])
        # print(smallerseq)
    elif counter == 8:
        if ((SF_Sort_arr[1][i])+5) == 5000:
            print(((SF_Sort_arr[1][i])+5))
            seq = seq+1
            sequence[0].append(seq)
            sequence[1].append(smallerseq)   
            sequence[2].append(smallerstateis)
            # sequence[0] = np.asarray(sequence[0])-1
            sequence[0] = [x-1 for x in sequence[0]]
            break
        # print(SF_Sort_arr[0][i+1])
        seq = seq+1
        # print(smallerseq)
        # print([(SF_Sort_arr[1][i])+5,(SF_Sort_arr[1][i+1])])
        smallerstateis.append([(SF_Sort_arr[1][i])+5,(SF_Sort_arr[1][i+1])])
        smallerseq.append(SF_Sort_arr[0][i+1])
        # print(smallerseq)
        # print(counter)
        sequence[0].append(seq)
        sequence[1].append(smallerseq)   
        sequence[2].append(smallerstateis)
        smallerseq = []
        smallerstateis = []
        counter  = 0

print((sequence[0]))
print(len(sequence))
print((sequence[1]))
if len(sequence[0]) == 14:
    print('hi')
    leftovers = [[sequence[0][-1]],[sequence[1][-1]],[sequence[2][-1]]]
    for i in range(len(sequence)):
        sequence[i].pop(-1)
else:
    leftovers = find_leftover(SF_Sort_arr)
    
        # sequence[i] = sequence[i].pop(-1)
# except Exception as e:
#     logger.error(run_num + ' failed during sequencing')
#     logger.exception(e)
print(counter)
# leftovers = [[sequence[0][-1]+1],[],[]]

# try:
def find_leftover(SFsort_info):
    left = [[sequence[0][-1]+1],[],[]]
    counter = 0
    for i in range((len(SFsort_info[1])-(np.mod((len(SFsort_info[1])), 8))), len(SFsort_info[1])-1):
        counter = counter+1
        if counter < 8:
            left[1].append(SFsort_info[0][i+1])
            left[2].append([(SFsort_info[1][i])+5,(SFsort_info[1][i+1])])
    return left
# except Exception as e:
#     logger.error(run_num + ' failed during leftovers sequencing')
#     logger.exception(e)
# leftovers = find_leftover(SF_Sort_arr)

print(leftovers)

print('sequences '+str(sequence[0]))
print('sequence order: '+str(sequence[1][0]))
# print(leftovers)


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
3
[[1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0]]
0
[[14], [1, 2, 3, 4, 5, 6], [[4700, 4740], [4745, 4785], [4790, 4830], [4835, 4875], [4880, 4920], [4925, 4965]]]
sequences [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
sequence order: [1, 2, 3, 4, 5, 6, 7, 0]


In [93]:
# In[20]:

# for ind in df.index:
#     print('transition: '+ str(df['nicknames'][ind]) + ' location: ' + str(df['transition_locations'][ind]))
#print(df['transition_locations'],df['nicknames'] )

# try:
## this could probably be prettier, but it seems to work...
def organize_SF(SFsort_info): ## sometimes pulse 0 has the state switch. In that case, need to account by if clauses below
    counter = 0
    seq = 0
    seq_arr = ([[],[],[]])
    smallerseq = []
    smallerstateis = []
    for i in range(len(SFsort_info[1])-(np.mod((len(SFsort_info[1])), 8))):  ##111 mod 8 = 7, so essentially 111-7 = 104
        counter = counter+1
        if counter < 8:
            if (SF_Sort_arr[1][i]) == 0:
                smallerstateis.append([(SFsort_info[1][i])+5,(SFsort_info[1][i+1])])
                smallerseq.append(SFsort_info[0][i+1])
                seq = seq+1
                continue
            smallerstateis.append([(SFsort_info[1][i])+5,(SFsort_info[1][i+1])])
            smallerseq.append(SFsort_info[0][i+1])
            # print(SF_Sort_arr[0][i+1])
            # print(smallerseq)
        elif counter == 8:
            if ((SF_Sort_arr[1][i])+5) == 5000:
                print(((SF_Sort_arr[1][i])+5))
                seq = seq+1
                seq_arr[0].append(seq)
                seq_arr[1].append(smallerseq)   
                seq_arr[2].append(smallerstateis)
                seq_arr[0] = [x-1 for x in seq_arr[0]] ## reset to sequences are 1-14 instead of 2-15
                break
            seq = seq+1
            smallerstateis.append([(SFsort_info[1][i])+5,(SFsort_info[1][i+1])])
            smallerseq.append(SFsort_info[0][i+1])
            seq_arr[0].append(seq)
            seq_arr[1].append(smallerseq)   
            seq_arr[2].append(smallerstateis)
            smallerseq = []
            smallerstateis = []
            counter  = 0
    return seq_arr

def find_leftover(SFsort_info, seq_arr):
    left = [[seq_arr[0][-1]+1],[],[]]
    counter = 0
    for i in range((len(SFsort_info[1])-(np.mod((len(SFsort_info[1])), 8))), len(SFsort_info[1])-1):
        counter = counter+1
        if counter < 8:
            left[1].append(SFsort_info[0][i+1])
            left[2].append([(SFsort_info[1][i])+5,(SFsort_info[1][i+1])])
    return left

#try:
sequence = organize_SF(SF_Sort_arr)
if len(sequence[0]) == 14:
    leftovers = [[sequence[0][-1]],[sequence[1][-1]],[sequence[2][-1]]]
    for i in range(len(sequence)):
        sequence[i].pop(-1)
else:
    leftovers = find_leftover(SF_Sort_arr, sequence)

# except Exception as e:
#     logger.error(run_num + ' failed during sequencing')
#     logger.exception(e)

print((sequence[0]))
print(len(sequence))
print((sequence[1]))
    
        # sequence[i] = sequence[i].pop(-1)
# leftovers = [[sequence[0][-1]+1],[],[]]

# except Exception as e:
#     logger.error(run_num + ' failed during leftovers sequencing')
#     logger.exception(e)
# leftovers = find_leftover(SF_Sort_arr)

print(leftovers)

print('sequences '+str(sequence[0]))
print('sequence order: '+str(sequence[1][0]))
# print(leftovers)


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
3
[[1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0]]
0
[[14], [1, 2, 3, 4, 5, 6], [[4700, 4740], [4745, 4785], [4790, 4830], [4835, 4875], [4880, 4920], [4925, 4965]]]
sequences [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
sequence order: [1, 2, 3, 4, 5, 6, 7, 0]


In [89]:
print(sequence[0])
print(sequence[1])
print(len(sequence))
print(len(sequence[0]))
print(len(sequence[1]))
print(len(sequence[2]))
print((sequence[2][-1]))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
[[1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0]]
3
13
13
13
[[4340, 4380], [4385, 4425], [4430, 4470], [4475, 4515], [4520, 4560], [4565, 4605], [4610, 4650], [4655, 4695]]


## t0 aligning ##

In [54]:
## use 6Li t0 for all instead of for themselves individually

start = time.time()

NaIthresh=2000
Li6thresh=1000

threshold_array = (np.full(len(ys_basesub), NaIthresh))
threshold_array[-1] = Li6thresh

# xValues=[]
# xtimems=[]

#@njit ## numba does not support reversed, but this could be changed if it's slow
def find_offset(ys, thresharr):
    xCrosses = np.zeros((len(ys), numRuns)) #outer array is crossing arrays for given channel, inner array is crossing for each event
    offset = np.zeros((len(ys), numRuns), dtype=np.int32) ##offset in bins for each channel, each pulse
    modeCrosses = np.zeros((len(ys)), dtype=np.float64)
    for i in reversed(range(len(ys))):
        #xValues.append([])
        for p in range(len(ys[i])):
            xing = np.argmax(ys[i][p] > thresharr[i])
            #print(xing)
            xCrosses[i][p] = xing
        modeCrosses[i] = (st.mode(xCrosses[i])) #find the most typical crossing value for each channel
    #     xValues[i].append(np.arange(len(ys_arr[i][0])+2*extendedRange)-modeCrosses[i])
    #     xtimems.append(xValues[i][-1]*resolution[i]*10**(-6))
        for p in range(len(xCrosses[i])):
            offset[i][p] = (modeCrosses[-1] - xCrosses[i][p]) ## make sure this is the correct sign!!! 
    if (np.all(xCrosses[-1])) == False:
        raise Exception('ERROR: 6Li threshold was not reached for at least one pulse')
    return offset, xCrosses, modeCrosses
                           
offset, xCrosses, modeCrosses = find_offset(ys_basesub, threshold_array)

print((xCrosses[0]))
print(np.std(xCrosses[0]))

print(len(offset))
print(len(offset[-1]))
print(offset[-1][2045:2055])
print(offset[-1][2053])

end = time.time()
print('finding offset time: ' + str(end-start))            

[1996. 1996. 1997. ... 1997. 1996. 1998.]
35.46794180665126
13
5000
[ 0 -1  1 -1  0 -1  0  1  2  0]
2
finding offset time: 13.3553626537323


In [55]:
## this cell loops through every channel as opposed to inputting all channels at once. 5x faster

start = time.time()

## extend all arrays by a value, check that the max number of offset on 6Li is less than that value ##
extendedRange = 3 ## must be a positive value which to extend ys_arr
if abs(max(offset[-1], key = abs)) > extendedRange: ## if the max offset of 6Li is >extendedRange, something is wrong
    raise Exception('ERROR: largest offset greater than extended range')

ys_ext = np.zeros((len(ys_basesub), len(ys_basesub[0]), len(ys_basesub[0][0])+extendedRange*2), dtype=np.float64)
ys_cut = np.zeros((len(ys_basesub), len(ys_basesub[0]), (len(ys_ext[0][0])-((extendedRange*2)+1)*2)))
xs_cut = np.zeros((len(ys_cut), len(ys_cut[0][0])))

# cant use jit because np.pad is not supported
def align_cut_norm(ys, xs_arr, extendedr):
    tempys_ext = np.zeros((len(ys), len(ys[0])+extendedr*2), dtype=np.float64)
    tempys_cut = np.zeros((len(ys), (len(tempys_ext[0])-((extendedr*2)+1)*2)))
    tempxs_cut = np.zeros(len(tempys_cut[0]))
    for p in range(len(ys)):
        #print(len(ys_basesub[i][p]))
        tempys_ext[p] = np.pad(ys[p], extendedr, 'constant', constant_values=(0))
        tempys_ext[p] = np.roll(tempys_ext[p],offset[-1][p]) ## assumes 6Li at -1 position
        #ys_cut[i][p] = ys_ext[i][p][((extendedRange*2)+1):-((extendedRange*2)+1)].copy() ## cut by 7 (if extRange == 3)
        tempys_cut[p] = tempys_ext[p][((extendedr*2)+1):-((extendedr*2)+1)].copy() ## cut by 7 (if extRange == 3)
        tempys_cut[p] = tempys_cut[p]/HeNorms[p] ## normalize by 3He integral
        #print(ys_ext[i][p][0])
        #print(ys_basesub[p][0:10])
    x_cut_amt = int((len(ys[0]) - len(tempys_cut[0]))/2)
    tempxs_cut = xs_arr[x_cut_amt:-x_cut_amt].copy()
    #print(len(tempxs_cut))
    return tempys_cut, tempxs_cut

for i in range(len(ys_basesub)):
    ys_cut[i], xs_cut[i] = align_cut_norm(ys_basesub[i], xs[i], extendedRange)

checkp = 2053
print(offset[-1][checkp]) ## checking offset for one example checkpulse
print('original index for checkpulse: '+str(np.argmax(ys_basesub[0][checkp]> 2000))) ## we can follow the index as it changes with extension/cut
#print('extended range index for checkpulse: '+str(np.argmax(ys_ext[0][checkp]> 2000)))
print('cut array index for checkpulse: '+str(np.argmax((ys_cut[0][checkp]*HeNorms[checkp])> 2000)))

print(len(offset))
print(len(offset[0]))
print(offset[-1][100:150])

print(len(ys_basesub[0][0]))
#print(len(ys_ext[0][0]))
print(len(ys_cut[0][0]))
# print(len(xs_cut[0]))

## del ys_ext ### get rid of this variable??
end = time.time()
print('aligning and cutting time: ' + str(end-start))            

2
original index for checkpulse: 1996
cut array index for checkpulse: 1994
13
5000
[ 0 -1  1  0  2  0  1  0  1 -1  0  1 -1  0 -1  0  1  2  0  1 -1  1 -1  1
  1  0  2  0 -1  1  0 -1  2  0  0 -1 -1  1  1  1 -1  2  1  0  0 -1 -1  0
  0  1]
9000
8992
aligning and cutting time: 16.428938388824463


In [24]:
## don't think I need this actually

# start = time.time()

# #@jit(nopython = True) ## Actually JIT seems to be slower here!
# def all_sums_normed(ys, baseR, numpoints): 
#     tempsums = np.zeros((len(ys), numRuns), dtype=np.float64)
#     for i in range((len(ys))):
#         for pulse in range((len(eventcount_arr[0]))): ## all have 5000 pulses
#             tempsums[i][pulse] = np.sum(ys[i][pulse][baseR+5:-1])
#             tempsums[i][pulse] = tempsums[i][pulse] / HeNorms[pulse]
#     return tempsums

# sums_normed = all_sums_normed(ys_cut, baseR, numSamples)
# print(sums)
# print(sums_normed)


# end = time.time()
# print('sums time: ' + str(end-start))            

## add up N number of pulses

In [60]:
## add up pulses for their respective state, in each 8 step sequence
## turning into a by channel function 06.13.24

start = time.time()

legend = ['NaI5 (downstream)','NaI4R (upstream)','6Li']

## t_to_eV = 0.5*(1.6749286*10**(-27))*((27.3/TIME))**2*(6.24151*10**18 [J/eV])

added_pulses = np.zeros((len(ys_cut), len(sequence[0]), 8, len(ys_cut[0][0])), dtype=np.float64) ## 13 sequences, 8 stages each works?
## i channels, 13 sequences each, 8 states each sequence, 8992 num points

ON_OFF_sums = np.zeros((len(ys_cut), len(sequence[0]), 2, len(ys_cut[0][0])), dtype=np.float64) ## 13 sequences, 2 for ON or OFF for each sequence

#@njit
def add_pulse(ys, SFarr):
    tempadded_p = np.zeros((len(SFarr[0]), 8, len(ys[0])), dtype=np.float64)    
    temp_ONOFF = np.zeros((len(SFarr[0]), 2, len(ys[0])), dtype=np.float64)
    for seq in range(0, len(SFarr[0])): ## for every sequence
        for state in range(0, len(SFarr[1][0])): ## for every state in the sequence
            print(state)
            for p in range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1])+1): ##not sure if this works. from 20-60 for example
                # print(((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1])+1))
#                 tempadded_pulses[i] = added_pulses[i]+np.array(ys_cut[i][j])
                tempadded_p[seq][state] = np.add(tempadded_p[seq][state],(ys[p])) ## start with zeros, add to each iteratively
            if state==0 or state==3 or state==5 or state==6: ## these are ON states
                temp_ONOFF[seq][0] = np.add(temp_ONOFF[seq][0],(tempadded_p[seq][state])) ## start with zeros, add to each iteratively
            if state==1 or state==2 or state==4 or state==7: ## these are OFF states
                temp_ONOFF[seq][1] = np.add(temp_ONOFF[seq][1],(tempadded_p[seq][state])) ## start with zeros, add to each iteratively
    return tempadded_p, temp_ONOFF
                
for i in range(len(ys_cut)):
    added_pulses[i], ON_OFF_sums[i] = add_pulse(ys_cut[i], sequence)
                
## plotting examples
plt.plot(xs_cut[i], added_pulses[0][0][0] , label=legend[0] +', sequence 1 state 1, 40 pulses added')
plt.plot(xs_cut[i], added_pulses[0][0][1] , label=legend[0] +', sequence 1 state 2, 40 pulses added')
plt.plot(xs_cut[i], added_pulses[0][1][0] , label=legend[0] +', sequence 2 state 1, 40 pulses added') 
plt.plot(xs_cut[i], added_pulses[0][1][1] , label=legend[0] +', sequence 2 state 2, 40 pulses added') 
    
plt.title('Detector signals') 
plt.xlabel("time from trigger (ns)")
plt.ylabel("ADC")

# plt.axvline(xs[0][baseL], ls = '--')
# plt.axvline(xs[0][baseR], ls = '--')
#plt.axvline(xs[1][intgrL], ls = '--', c ='g')
#plt.axvline(xs[1][intgrR], ls = '--', c ='g')
#plt.axvline(xs[2][HeintgrL], ls = '--', c ='r')
#plt.axvline(xs[2][HeintgrR], ls = '--', c ='r')

plt.legend()
plt.show()

print(len(added_pulses))
print(len(added_pulses[1]))

end = time.time()
print('summing pulses into their states time: ' + str(end-start))  

0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3


In [75]:
start = time.time()
print(Asym[0])

Asym = np.zeros((len(ON_OFF_sums), len(ON_OFF_sums[0][0][0])), dtype=np.float64) ## 1 Asym for each channel, not for each sequence (can change)

def asym(ON_OFF_arr):
    tempasym = np.zeros((len(ON_OFF_arr[0][0])), dtype=np.float64)
    for seq in range(len(ON_OFF_arr[0])): ## number of sequences
        asymform = ((ON_OFF_arr[seq][0]-ON_OFF_arr[seq][1]) / (ON_OFF_arr[seq][0]+ON_OFF_arr[seq][1]))
        tempasym = np.add(asymform,tempasym)
    normedasym = tempasym/len(ON_OFF_sums[0])
    return normedasym

for i in range(len(ON_OFF_sums)):
    Asym[i] = asym(ON_OFF_sums[i])
#Asym = (ON_OFF_sums[-1][0][0]-ON_OFF_sums[-1][0][1]) / (ON_OFF_sums[0][0][0]+ON_OFF_sums[0][0][1])
print(len(Asym))
print(len(Asym[0]))
print(Asym[0])

end = time.time()
print('calculate asymmetry time: ' + str(end-start)) 

[ 0.01009674  0.07850341 -0.00518195 ... -0.00663549  0.00263154
 -0.0339125 ]
13
8992
[ 0.01009674  0.07850341 -0.00518195 ... -0.00663549  0.00263154
 -0.0339125 ]
calculate asymmetry time: 0.0019979476928710938


In [86]:
np.save(AddedPulseSavename, added_pulses)
np.save(AsymSavename, added_pulses)

fullend = time.time()
print('full processing time: ' + str(fullend-fullstart))  

In [77]:
## plotting examples
# Asym = (ON_OFF_sums[-1][0][0]-ON_OFF_sums[-1][0][1]) / (ON_OFF_sums[0][0][0]+ON_OFF_sums[0][0][1])
# plt.plot(xs_cut[i], ON_OFF_sums[0][0][0] , label=legend[0] +', sequence 1 ON pulses')
# plt.plot(xs_cut[i], ON_OFF_sums[0][0][1] , label=legend[0] +', sequence 1 OFF pulses')
# plt.plot(xs_cut[i], ON_OFF_sums[0][1][0] , label=legend[0] +', sequence 2 ON pulses') 
# plt.plot(xs_cut[i], ON_OFF_sums[0][1][1] , label=legend[0] +', sequence 2 OFF pulses') 
plt.plot(xs_cut[2], Asym[2] , label=legend[0] +'NaI asym') 

    
#plt.xlim(-50000, 900000)
plt.ylim(-1.5,1.5)
plt.title('Detector signals') 
plt.xlabel("time from trigger (ns)")
plt.ylabel("ADC")

# plt.axvline(xs[0][baseL], ls = '--')
# plt.axvline(xs[0][baseR], ls = '--')
#plt.axvline(xs[1][intgrL], ls = '--', c ='g')
#plt.axvline(xs[1][intgrR], ls = '--', c ='g')
#plt.axvline(xs[2][HeintgrL], ls = '--', c ='r')
#plt.axvline(xs[2][HeintgrR], ls = '--', c ='r')

#plt.text(1.5e7, 1200, 'test', fontsize=12)
plt.legend()
plt.show()

In [81]:
dftest = pd.DataFrame(Asym)
print(dftest)

        0          1         2         3         4          5          6     \
0   0.010097   0.078503 -0.005182 -0.640061  0.039544  -0.958567   0.229285   
1   0.086791   0.392966  0.068683 -5.338004 -0.079736  -0.753386   2.097498   
2   0.514255  -0.201895  0.004301  0.445483  0.045710   0.291672  -0.291194   
3  -0.678252 -22.803517 -0.489773  0.161298  0.100997   0.002159  -0.030375   
4  -0.038016  -0.282601 -0.100679  0.078909 -0.537298  12.363887   1.059083   
5   0.536799  -0.109452 -0.076119 -0.273232  0.470396  -0.133351  -0.799243   
6   0.412642   0.625526  0.038799  0.065525  0.459505   0.022234   0.021983   
7  -0.635317  -0.000412  0.130328  1.937651  0.162294  -0.264599   0.216655   
8   0.020348   0.097881 -0.044265  1.427839 -0.248716  -0.130505  -0.167178   
9  -0.146590   0.148398  0.249473  0.274707  0.117211  -0.072060  -0.041347   
10 -0.047327  -0.009126  0.077373  0.521125 -0.317845   0.126426  17.376556   
11  0.016673  -0.057803  0.213716  1.254545  0.01905

In [96]:
# print(len(added_pulses)) ## num channels
# print(len(added_pulses[0])) ## num sequences
# print(len(added_pulses[0][0])) ## num states in a sequence
# print(len(added_pulses[0][0][0])) ## num points in a statedata

# for seq in range(0, len(sequence[0])): ## for every sequence
#     for state in range(0, len(sequence[1][0])): ## for every state in the sequence
#         print(sequence[2][seq][state][0])
#         print((sequence[2][seq][state][1])+1) ##not sure if this works. from 20-60 for example
# #                 tempadded_pulses[i] = added_pulses[i]+np.array(ys_cut[i][j])

3
13
8
8992
20
61
65
106
110
151
155
196
200
241
245
286
290
331
335
376
380
421
425
466
470
511
515
556
560
601
605
646
650
691
695
736
740
781
785
826
830
871
875
916
920
961
965
1006
1010
1051
1055
1096
1100
1141
1145
1186
1190
1231
1235
1276
1280
1321
1325
1366
1370
1411
1415
1456
1460
1501
1505
1546
1550
1591
1595
1636
1640
1681
1685
1726
1730
1771
1775
1816
1820
1861
1865
1906
1910
1951
1955
1996
2000
2041
2045
2086
2090
2131
2135
2176
2180
2221
2225
2266
2270
2311
2315
2356
2360
2401
2405
2446
2450
2491
2495
2536
2540
2581
2585
2626
2630
2671
2675
2716
2720
2761
2765
2806
2810
2851
2855
2896
2900
2941
2945
2986
2990
3031
3035
3076
3080
3121
3125
3166
3170
3211
3215
3256
3260
3301
3305
3346
3350
3391
3395
3436
3440
3481
3485
3526
3530
3571
3575
3616
3620
3661
3665
3706
3710
3751
3755
3796
3800
3841
3845
3886
3890
3931
3935
3976
3980
4021
4025
4066
4070
4111
4115
4156
4160
4201
4205
4246
4250
4291
4295
4336
4340
4381
4385
4426
4430
4471
4475
4516
4520
4561
4565
4606
4610
4651
4655

# old summing #

In [33]:
# for ind in df.index:
#     print('transition: '+ str(df['nicknames'][ind]) + ' location: ' + str(df['transition_locations'][ind]))
#print(df['transition_locations'],df['nicknames'] )

counter = 0
sequence = [[],[],[]]
seq = 0
smallerseq = []
smallersum = []
    
for i in range(len(SF_Sort_arr[1])-(np.mod((len(SF_Sort_arr[1])), 8))):  ##111 mod 8 = 7, so essentially 111-7 = 104
    counter = counter+1
    #print( i )
    #print(counter)
    if counter < 8:
        smallersum.append(np.sum(sums_normed[0][(SF_Sort_arr[1][i])+5:(SF_Sort_arr[1][i+1])])) ## minus first 5 to get rid of transitioning pulses
        smallerseq.append(SF_Sort_arr[0][i+1])
        #print(smallerseq)
    elif counter == 8:
        seq = seq+1
        smallersum.append(np.sum(sums_normed[0][(SF_Sort_arr[1][i])+5:(SF_Sort_arr[1][i+1])]))
        smallerseq.append(SF_Sort_arr[0][i+1])
        sequence[0].append(seq)
        sequence[1].append(smallerseq)   
        sequence[2].append(smallersum)
        smallerseq = []
        smallersum = []
        counter  = 0
        #print(test_arr[1][i+1])
        
        
# print(counter)
# print(len(testsumarr))
#print(len(testsumarr))
# print(len(sequence))
# print(len(sequence[0]))
# print(len(sequence[1]))

print(sequence)
print(len(sequence))
print(len(sequence[0]))
print(len(sequence[1]))
print(len(sequence[2]))

# # print((testsumarr))
# print(np.sum(testsums[0][35:75]))
# print(len(np.ndarray.flatten(np.asarray(sequence[1]))))

leftoversum = []
leftovers = [[sequence[0][-1]+1],[],[]]

for i in range((len(SF_Sort_arr[1])-(np.mod((len(SF_Sort_arr[1])), 8))), len(SF_Sort_arr[1])-1):
    counter = counter+1
    print( i )
    #print( i )
    #print(counter)
    if counter < 8:
        leftovers[1].append(SF_Sort_arr[0][i+1])
        leftovers[2].append(np.sum(sums_normed[0][(SF_Sort_arr[1][i])+5:(SF_Sort_arr[1][i+1])])) ## minus first 5 to get rid of transitioning pulses
    elif counter == 8: ## not really needed here, maybe delete
        seq = seq+1
        leftoversum.append(np.sum(sums_normed[0][(SF_Sort_arr[1][i])+5:(SF_Sort_arr[1][i+1])]))
        smallerseq.append(SF_Sort_arr[0][i+1])
        sequence[0].append(seq)
        sequence[1].append(smallerseq)       
        smallerseq = []
        counter  = 0
        #print(test_arr[1][i+1])

print(leftovers)
print(leftoversum)

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], [[1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 3, 4, 5, 6, 7, 0]], [[438560.3977175717, 437459.03691159806, 437887.9796430829, 438994.13954999216, 434730.32151408633, 436904.0810749381, 438280.35365632904, 438582.48215183715], [438363.2158272527, 437854.6807106178, 440413.1473017337, 438099.31027938856, 437182.3204206178, 438591.2003248631, 436944.2384369202, 437814.37876274146], [435967.6903801842, 437897.2277495987, 437486.28856541484, 436446.2405218859, 433973.6628319679, 437976.8829429539, 438938.95632722444, 435791.5491605206], [437927.44357607065, 440899.2729838694, 439685.5801664869, 439136.5752539784, 437610.2906584542, 438397.6515085819, 437811.88403374865, 441572

In [42]:
## not sure if dataframe is actually useful, but helpful to visualize
cols = ['sequence', 'SF states', 'sums per state']

df_SumsOrg = pd.DataFrame({cols[0]: sequence[0],            
                   cols[1]: sequence[1],
                   cols[2]: sequence[2]})

df_LO = pd.DataFrame({cols[0]: leftovers[0][0], ## LeftOvers dataframe, not sure if it will be used           
                   cols[1]: leftovers[1],
                   cols[2]: leftovers[2]})

# df_SumsOrg.append(df_LO)
print('organized sums:')
print(df_SumsOrg)

print('leftovers:')
print(df_LO)

organized sums:
    sequence                 SF states  \
0          1  [1, 2, 3, 4, 5, 6, 7, 0]   
1          2  [1, 2, 3, 4, 5, 6, 7, 0]   
2          3  [1, 2, 3, 4, 5, 6, 7, 0]   
3          4  [1, 2, 3, 4, 5, 6, 7, 0]   
4          5  [1, 2, 3, 4, 5, 6, 7, 0]   
5          6  [1, 2, 3, 4, 5, 6, 7, 0]   
6          7  [1, 2, 3, 4, 5, 6, 7, 0]   
7          8  [1, 2, 3, 4, 5, 6, 7, 0]   
8          9  [1, 2, 3, 4, 5, 6, 7, 0]   
9         10  [1, 2, 3, 4, 5, 6, 7, 0]   
10        11  [1, 2, 3, 4, 5, 6, 7, 0]   
11        12  [1, 2, 3, 4, 5, 6, 7, 0]   
12        13  [1, 2, 3, 4, 5, 6, 7, 0]   

                                       sums per state  
0   [438560.3977175717, 437459.03691159806, 437887...  
1   [438363.2158272527, 437854.6807106178, 440413....  
2   [435967.6903801842, 437897.2277495987, 437486....  
3   [437927.44357607065, 440899.2729838694, 439685...  
4   [437347.70291982073, 435075.2823116139, 439091...  
5   [437809.7177920478, 438776.9507144898, 438024....  
6  

In [71]:
ONsum = 0
OFFsum = 0

for seq in range(len(sequence[0])):
#     print('sequence: ' + str(seq))
    for state in range(len(sequence[1][seq])):
        if sequence[1][seq][state]==0 or sequence[1][seq][state]== 3 or sequence[1][seq][state]==5 or sequence[1][seq][state]==6:
#             print(sequence[1][seq][state])
            ONsum = ONsum+sequence[2][seq][state]
        if sequence[1][seq][state]==1 or sequence[1][seq][state]== 2 or sequence[1][seq][state]==4 or sequence[1][seq][state]==7:
            OFFsum = OFFsum+sequence[2][seq][state]

# # print(range(len(sequence[0])))
# print(len(sequence[1]))
# print(len(sequence[1][0]))
# print((sequence[1][0][2]))

print(ONsum)
print(OFFsum)

print(ONsum - OFFsum)
print(OFFsum)
Asym = (ONsum - OFFsum)/(ONsum + OFFsum)
print(Asym)


22739918.541138384
22741826.78821649
-1908.2470781058073
22741826.78821649
-4.1956329166510336e-05


## Useful plotting below ##

In [27]:
# ## plotting sums distribution, examples for thesis

# NaI5 = 5
# NaI19 = 7
# a = sums_normed[-1]
# _ = plt.hist(a, bins=(np.arange(min(a)-3,max(a)+3,1)))  # arguments are passed to np.histogram

# plt.title("Distribution of Sums for One La run, 6Li detector")
# plt.xlabel("Integral")
# plt.ylabel("Counts")
# plt.show()

In [9]:
## simple plotting ##
## examples for thesis

# pulse = 2048

# # plt.plot(xs[1],ys_arr[1][pulse], label='NaI')
# # # plt.plot(xs[1],ys_basesub[0][pulse], label='NaI') 
# # plt.plot(xs[-1],ys_arr[-1][pulse], label='6Li') 
# NaI1 = 0
# NaI2 = 7

# plt.plot(xs[NaI1],ys_basesub[NaI1][pulse], label='NaI detector ' + str(chan_enab[NaI1]))
# plt.plot(xs[NaI2],ys_basesub[NaI2][pulse], label='NaI detector ' + str(chan_enab[NaI2]))
# plt.plot(xs[-1],ys_basesub[-1][pulse], label='6Li (inverted signal)') 

# plt.title('Example Pulses (Pr sample)') 
# plt.xlabel("time from trigger (ns)")
# plt.ylabel("ADC")
# #print(len(ys_arrHe[-1][0]))

# # plt.axvline(xs[0][baseRHe], ls = '--', c ='r')
# # plt.axvline(xs[0][Heregion[0]], ls = '--')
# # plt.axvline(xs[0][Heregion[1]], ls = '--')
# #plt.axvline(xs[1][-1], ls = '--', c ='g')
# #plt.axhline(4060, ls = '--')
# #plt.axvline(xs[0][17000], ls = '--', c ='r')

# #plt.text(1.5e7, 1200, 'test', fontsize=12)
# plt.legend()
# plt.show()

In [10]:
## determine t0, and plot the std and outlier based on the channel ##
## this clearly shows that 6Li should be the standard for t0 ##
## examples for thesis

start = time.time()

NaIthresh=2000
Li6thresh=1000

threshold_array = (np.full(len(ys_basesub), NaIthresh))
threshold_array[-1] = Li6thresh
#print(threshold_array)

xCrosses = np.zeros((len(ys_basesub), numRuns)) #outer array is crossing arrays for given channel, inner array is crossing for each event
modeCrosses=[] #single array with one value per channel
xValues=[]
xtimems=[]
offset = np.zeros((len(ys_basesub), numRuns), dtype=np.int32) ##offset in bins for each channel, each pulse
extendedRange=3

for i in (range(len(ys_basesub))):
    xValues.append([])
    for p in range(len(ys_basesub[i])):
        xing = np.argmax(ys_basesub[i][p] > threshold_array[i])
        #print(xing)
        xCrosses[i][p] = xing
    modeCrosses.append(st.mode(xCrosses[i])) #find the most typical crossing value for each channel
#     xValues[i].append(np.arange(len(ys_arr[i][0])+2*extendedRange)-modeCrosses[i])
#     xtimems.append(xValues[i][-1]*resolution[i]*10**(-6))
    for p in range(len(xCrosses[i])):
        offset[i][p] = (modeCrosses[i] - xCrosses[i][p]) ## make sure this is the correct sign!!!

outliers = []
bindif = 4 ## set how strict your # of bins deviation for outliers

for i in range(len(xCrosses)):
    outliers.append([])
    for p in range(len(xCrosses[i])):
        if offset[i][p] > 3:
            outliers[i].append(p)
    print('std for channel ' + str(chan_enab[i]) + ': '+ str(np.std(offset[i])))
    #print(np.max(offset[i]))

print(outliers)
print(np.argwhere(offset[0] > 1713.0))

choff = -1 ## which channel to look at for histogram
a = offset[choff]
#print(a)

_ = plt.hist(a, bins=(np.arange(min(a)-3,max(a)+3,1)))  # arguments are passed to np.histogram

plt.title("Distribution of offsets from each detector's respective mode: 6Li")
plt.show()

end = time.time()
print(end-start)            

KeyboardInterrupt: 

In [23]:
# ## pulse 150 shows max offset, pulse 2048 shows example false trigger
# ## plotting examples for thesis

# pulse = 150

# xs_cut = np.zeros((len(ys_cut), len(ys_cut[0][0]))) ## take extended ys array and cut off ends 
 
# NaI1 = 0
# NaI2 = 7

# plt.plot(xs[NaI1],ys_basesub[NaI1][pulse], label='NaI detector ' + str(chan_enab[NaI1]) + ', before alignment')
# # plt.plot(xs[NaI2],ys_basesub[NaI2][pulse], label='NaI detector ' + str(chan_enab[NaI2]))
# #plt.plot(xs[-1],ys_basesub[-1][pulse], label='6Li (inverted signal)') 
# plt.plot(xs[-1],ys_basesub[-1][pulse], label='6Li' + ', before alignment') 

# plt.plot(xs_cut[NaI1],ys_cut[NaI1][pulse], label='NaI detector ' + str(chan_enab[NaI1]) + ', after alignment')
# # plt.plot(xs[NaI2],ys_basesub[NaI2][pulse], label='NaI detector ' + str(chan_enab[NaI2]))
# #plt.plot(xs[-1],ys_basesub[-1][pulse], label='6Li (inverted signal)') 
# plt.plot(xs_cut[-1],ys_cut[-1][pulse], label='6Li' + ', after alignment') 

# plt.axhline(NaIthresh, ls = '--', c ='#1f77b4', label='NaI t0 threshold')
# plt.axhline(Li6thresh, ls = '--', c ='#ff7f0e', label='6Li t0 threshold')
# plt.axvline(xs_cut[0][baseR+5], ls = '--', c ='b', label='begin integral')
# plt.axvline(xs[0][baseR+5], ls = '--', c ='r', label='old begin integral')

# plt.title('Example Pulses (Pr sample)') 
# plt.xlabel("time from trigger (ns)")
# plt.ylabel("ADC")

# plt.legend()
# plt.show()

12
9000
12
8992
8992


## previous testing ##

In [21]:
## old array, loads all channels at once, slower

## extend all arrays by a value, check that the max number of offset on 6Li is less than that value ##
start = time.time()

extendedRange = 3 ## must be a positive value which to extend ys_arr
if abs(max(offset[-1], key = abs)) > extendedRange: ## if the max offset of 6Li is >extendedRange, something is wrong
    raise Exception('ERROR: largest offset greater than extended range')

# cant use jit because np.pad is not supported
def align_cut_norm(ys, xs_arr, extendedr):
    ys_ext = np.zeros((len(ys), len(ys[0]), len(ys[0][0])+extendedr*2), dtype=np.float64)
    tempys_cut = np.zeros((len(ys), len(ys[0]), (len(ys_ext[0][0])-((extendedr*2)+1)*2)))
    tempxs_cut = np.zeros((len(tempys_cut), len(tempys_cut[0][0])))
    for i in range(len(ys)):
        for p in range(len(ys[i])):
            #print(len(ys_basesub[i][p]))
            ys_ext[i][p] = np.pad(ys[i][p], extendedr, 'constant', constant_values=(0))
            ys_ext[i][p] = np.roll(ys_ext[i][p],offset[-1][p]) ## assumes 6Li at -1 position
            #ys_cut[i][p] = ys_ext[i][p][((extendedRange*2)+1):-((extendedRange*2)+1)].copy() ## cut by 7 (if extRange == 3)
            tempys_cut[i][p] = ys_ext[i][p][((extendedr*2)+1):-((extendedr*2)+1)].copy() ## cut by 7 (if extRange == 3)
            tempys_cut[i][p] = tempys_cut[i][p]/HeNorms[p] ## normalize by 3He integral
            #print(ys_ext[i][p][0])
            #print(ys_basesub[p][0:10])
        x_cut_amt = int((len(ys[i][0]) - len(tempys_cut[i][0]))/2)
        tempxs_cut[i] = xs_arr[i][x_cut_amt:-x_cut_amt].copy()
    return tempys_cut, tempxs_cut

ys_cut, xs_cut = align_cut_norm(ys_basesub, xs, extendedRange)

checkp = 2053
print(offset[-1][checkp]) ## checking offset for one example checkpulse
print('original index for checkpulse: '+str(np.argmax(ys_basesub[0][checkp]> 2000))) ## we can follow the index as it changes with extension/cut
#print('extended range index for checkpulse: '+str(np.argmax(ys_ext[0][checkp]> 2000)))
print('cut array index for checkpulse: '+str(np.argmax((ys_cut[0][checkp]*HeNorms[checkp])> 2000)))

print(len(offset))
print(len(offset[0]))
print(offset[-1][100:150])

print(len(ys_basesub[0][0]))
#print(len(ys_ext[0][0]))
print(len(ys_cut[0][0]))
print(len(xs_cut[0]))

## del ys_ext ### get rid of this variable??
end = time.time()
print('aligning and cutting time: ' + str(end-start))            

2
original index for checkpulse: 1996
cut array index for checkpulse: 1994
13
5000
[ 0 -1  1  0  2  0  1  0  1 -1  0  1 -1  0 -1  0  1  2  0  1 -1  1 -1  1
  1  0  2  0 -1  1  0 -1  2  0  0 -1 -1  1  1  1 -1  2  1  0  0 -1 -1  0
  0  1]
9000
8992
8992
aligning and cutting time: 27.503201007843018


In [11]:
## trying pad without pad 

ys_ext_test = np.zeros((len(ys_basesub), len(ys_basesub[0]), len(ys_basesub[0][0])+3*2), dtype=np.float64)
ys_ext_test[0][0] = np.pad(ys_basesub[0][0], 3, 'constant', constant_values=(0))
print(ys_ext_test[0][0][0:5], ys_ext_test[0][0][-5:])
print(ys_basesub[0][0][0:5], ys_basesub[0][0][-5:])

def pad2(mat, padding, pad_with=0):
    n_rows = len(mat)
    # new empty matrix of the required size
    new_mat = [pad_with for row in range(n_rows + padding * 2)]
    
    #"insert" original matix in the empty matrix
    for row in range(n_rows):
        new_mat[row + padding] = mat[row]

    return new_mat

# test = [-1,-2,1,2,3,4,5]
# newtest = pad2(test, 3)
# print(test)
# print(newtest)

ys_ext_test2 = np.zeros((len(ys_basesub), len(ys_basesub[0]), len(ys_basesub[0][0])+3*2), dtype=np.float64)
# ys_ext_test2[0][0] = np.pad(ys_basesub[0][0], 3, 'constant', constant_values=(0))
ys_ext_test2[0][0] = pad2(ys_basesub[0][0], 3)
print(ys_ext_test2[0][0][0:5], ys_ext_test2[0][0][-5:])
print(ys_basesub[0][0][0:5], ys_basesub[0][0][-5:])

## Looks like it works!! 

[ 0.          0.          0.         -6.53071429 -5.53071429] [ 6.46928571 -0.53071429  0.          0.          0.        ]
[-6.53071429 -5.53071429 -5.53071429 -4.53071429 -6.53071429] [105.46928571  41.46928571  28.46928571   6.46928571  -0.53071429]
[ 0.          0.          0.         -6.53071429 -5.53071429] [ 6.46928571 -0.53071429  0.          0.          0.        ]
[-6.53071429 -5.53071429 -5.53071429 -4.53071429 -6.53071429] [105.46928571  41.46928571  28.46928571   6.46928571  -0.53071429]


In [8]:
## same as other align_cut but replacing np.pad with pad2, then using jit. Seems slower! Kept in case its needed

start = time.time()

extendedRange = 3 ## must be a positive value which to extend ys_arr
if abs(max(offset[-1], key = abs)) > extendedRange: ## if the max offset of 6Li is >extendedRange, something is wrong
    raise Exception('ERROR: largest offset greater than extended range')

@njit
def pad2(mat, padding, pad_with=0):
    n_rows = len(mat)
    # new empty matrix of the required size
    new_mat = [pad_with for row in range(n_rows + padding * 2)]
    
    #"insert" original matix in the empty matrix
    for row in range(n_rows):
        new_mat[row + padding] = mat[row]

    return new_mat

# cant use jit because np.pad is not supported
@njit
def align_cut2(ys, extendedr):
    ys_ext = np.zeros((len(ys), len(ys[0]), len(ys[0][0])+extendedr*2), dtype=np.float64)
    ys_cut = np.zeros((len(ys), len(ys[0]), (len(ys_ext[0][0])-((extendedr*2)+1)*2)))
    for i in range(len(ys)):
        for p in range(len(ys[i])):
            #print(len(ys_basesub[i][p]))
            ys_ext[i][p] = pad2(ys[i][p], extendedr)
            ys_ext[i][p] = np.roll(ys_ext[i][p],offset[-1][p]) ## assumes 6Li at -1 position
            #ys_cut[i][p] = ys_ext[i][p][((extendedRange*2)+1):-((extendedRange*2)+1)].copy() ## cut by 7 (if extRange == 3)
            ys_cut[i][p] = ys_ext[i][p][((extendedr*2)+1):-((extendedr*2)+1)].copy() ## cut by 7 (if extRange == 3)
            #print(ys_ext[i][p][0])
            #print(ys_basesub[p][0:10])
                
    return ys_cut

ys_cut2 = align_cut2(ys_basesub, extendedRange)

checkp = 2045
print(offset[-1][checkp]) ## checking offset for one example checkpulse
print('original index for checkpulse: '+str(np.argmax(ys_basesub[0][checkp]> 2000))) ## we can follow the index as it changes with extension/cut
#print('extended range index for checkpulse: '+str(np.argmax(ys_ext[0][checkp]> 2000)))
print('cut array index for checkpulse: '+str(np.argmax(ys_cut[0][checkp]> 2000)))

print(len(offset))
print(len(offset[0]))
print(offset[-1][100:150])

print(len(ys_basesub[0][0]))
#print(len(ys_ext[0][0]))
print(len(ys_cut[0][0]))

## del ys_ext ### get rid of this variable??

end = time.time()
print('aligning and cutting time: ' + str(end-start))            


KeyboardInterrupt



In [31]:
## add up pulses for their respective state, in each 8 step sequence
## old non-function

start = time.time()

legend = ['NaI5 (downstream)','NaI4R (upstream)','6Li']

## t_to_eV = 0.5*(1.6749286*10**(-27))*((27.3/TIME))**2*(6.24151*10**18 [J/eV])

added_pulses = np.zeros((len(ys_cut), len(sequence[0]), 8, len(ys_cut[0][0])), dtype=np.float64) ## 13 sequences, 8 stages each works?
## i channels, 13 sequences each, 8 states each sequence, 8992 num points

ON_OFF_sums = np.zeros((len(ys_cut), len(sequence[0]), 2, len(ys_cut[0][0])), dtype=np.float64) ## 13 sequences, 2 for ON or OFF for each sequence

for i in (range(len(ys_cut))): ## for every channel
#     added_pulses.append(np.zeros(len(ys_cut[i][0]))) ## make zeros array of length equal to window of cahnnel 
    for seq in range(0, len(sequence[0])): ## for every sequence
        for state in range(0, len(sequence[1][0])): ## for every state in the sequence
            for p in range((sequence[2][seq][state][0]),(sequence[2][seq][state][1])+1): ##not sure if this works. from 20-60 for example
#                 tempadded_pulses[i] = added_pulses[i]+np.array(ys_cut[i][j])
                added_pulses[i][seq][state] = np.add(added_pulses[i][seq][state],(ys_cut[i][p])) ## start with zeros, add to each iteratively
            if state==0 or state==3 or state==5 or state==6: ## these are ON states
                ON_OFF_sums[i][seq][0] = np.add(ON_OFF_sums[i][seq][0],(added_pulses[i][seq][state])) ## start with zeros, add to each iteratively
            if state==1 or state==2 or state==4 or state==7: ## these are OFF states
                ON_OFF_sums[i][seq][1] = np.add(ON_OFF_sums[i][seq][1],(added_pulses[i][seq][state])) ## start with zeros, add to each iteratively

## plotting examples
plt.plot(xs_cut[i], added_pulses[0][0][0] , label=legend[0] +', sequence 1 state 1, 40 pulses added')
plt.plot(xs_cut[i], added_pulses[0][0][1] , label=legend[0] +', sequence 1 state 2, 40 pulses added')
plt.plot(xs_cut[i], added_pulses[0][1][0] , label=legend[0] +', sequence 2 state 1, 40 pulses added') 
plt.plot(xs_cut[i], added_pulses[0][1][1] , label=legend[0] +', sequence 2 state 2, 40 pulses added') 
    
plt.title('Detector signals') 
plt.xlabel("time from trigger (ns)")
plt.ylabel("ADC")

# plt.axvline(xs[0][baseL], ls = '--')
# plt.axvline(xs[0][baseR], ls = '--')
#plt.axvline(xs[1][intgrL], ls = '--', c ='g')
#plt.axvline(xs[1][intgrR], ls = '--', c ='g')
#plt.axvline(xs[2][HeintgrL], ls = '--', c ='r')
#plt.axvline(xs[2][HeintgrR], ls = '--', c ='r')

plt.legend()
plt.show()

print(len(added_pulses))
print(len(added_pulses[1]))

end = time.time()
print('summing pulses into their states time: ' + str(end-start))  

13
13
summing pulses into their states time: 0.5340218544006348


In [107]:
NaIthresh=2000
Li6thresh=1000
print(len(ys_basesub))

threshold_array = (np.full(len(ys_basesub), NaIthresh))
threshold_array[-1] = Li6thresh

#xCrosses=[]#outer array is crossing arrays for given channel, inner array is crossing for each event
xCrosses = np.zeros((len(ys_basesub), numRuns))
modeCrosses=[] #single array with one value per channel
xValues=[]
xtimems=[]
extendedRange=3

for i in range(0,len(ys_basesub)):
#     xCrosses.append([])
    xValues.append([])
    for j in ys_basesub[i]:
        thresh_xing = np.diff(j > threshold_array[i], prepend=False)
        #print(len(thresh_xing))
        xing = np.argwhere(thresh_xing)[:,0]
        #print(xing[0])
        #print(len(xing))
    for p in range(len(ys_basesub[i])):
        #print(xing[0])
        xCrosses[i][p] = xing[0]
        #print(len(xCrosses[i]))
    modeCrosses.append(st.mode(xCrosses[i])) #find the most typical crossing value for each channel
    xValues[i].append(np.arange(len(ys_arr[i][0])+2*extendedRange)-modeCrosses[i])
    xtimems.append(xValues[i][-1]*resolution[i]*10**(-6))

print(len(xtimems[1]))
print(modeCrosses)
print(xValues[0])
print(len(xValues))
print(len(xCrosses))
print(len(xCrosses[0]))
print((xCrosses[0][10:30]))

12
9006
[1997.0, 1997.0, 1997.0, 1997.0, 1997.0, 1997.0, 1997.0, 1997.0, 1997.0, 1997.0, 1997.0, 1997.0]
[array([-1997., -1996., -1995., ...,  7006.,  7007.,  7008.])]
12
12
5000
[1997. 1997. 1997. 1997. 1997. 1997. 1997. 1997. 1997. 1997. 1997. 1997.
 1997. 1997. 1997. 1997. 1997. 1997. 1997. 1997.]


## Previous cells

In [9]:
### TESTING CELL ###
print(chan_enab)
print(len(chan_enab))
print(numSamples)
numRuns = int((fileLength[0]-20-numSamples[0])/(numSamples[0]+6)+1)
print(numRuns)
testys = np.zeros((len(chan_enab), numRuns,numSamples[0]), dtype=np.uint16)
print(len(testys[0][0]))
print(len(read_data))
print(len(read_data[0]))
print((read_data[0][45]))

numSamples[0]=((read_data[0][11]&0x00FF)<<16)+read_data[0][10]
print(numSamples[0])
print(testys[0][0][8999])
testys[0][0][8999] = read_data[0][8999]
print(testys[0][0][8999])
# print('He:')
# numSamplesHe = numSamples[-1]
# print(numRunsHe)
# ys_arrHe = np.zeros((1,numRuns,numSamples[-1]), dtype=np.uint16)
# print(len(ys_arrHe[0][0]))

# for i in range(0,len(chan_enab)):
#     print(numSamples[i])
#     if chan_enab[i] == 25:
#             print('3He channel loaded')

[ 0  1  2  3  4  5 19 20 21 22 23 24]
12
[9000 9000 9000 9000 9000 9000 9000 9000 9000 9000 9000 9000]
5000
9000
12
45030014
592
9000
0
595


In [18]:
import numba
from numba.typed import List
from numba import uint16, intc, uint32
from array import array
start=time.time()

#ys_arr = []
#ETTT_arr = []
#eventcount_arr = []
#ys_arr = List.empty_list(uint16)
#ETTT_arr = List.empty_list(intc)
#eventcount_arr = List.empty_list(intc)
#print(type(ETTT_arr[0]))

#ys_arr = List([np.empty(0) for _ in range(len(chan_enab))])
#ETTT_arr = List([np.empty(0) for _ in range(len(chan_enab))])
#eventcount_arr = List([np.empty(0) for _ in range(len(chan_enab))])
#def __init__(self):
#    self.ys_arr = typed.List.empty_list(types.float64)
#    return ys_arr
#print(eventcount_arr)

#@jit("uint16[:,:](uint16[:,:], uint32, uint32, uint32)")
@njit
def test2(read_data, channels, fileLength, numSamples):
    #ys_arr = []
    numRuns = int((fileLength[0]-20-numSamples[0])/(numSamples[0]+6)+1)
    ys_arr = np.zeros((len(channels), numRuns,numSamples[0]), dtype=np.uint16)
    ETTT_arr = np.zeros((len(channels), numRuns), dtype=np.intc)
    eventcount_arr = np.zeros((len(channels), numRuns), dtype=np.intc)
#     if channels[-1] != 25:
#         print('No 3He. Cannot normalize')
#         ys_arr = [0]
#         ETTT_arr = [0]
#         eventcount_arr = [0]
#         return ys_arr, ETTT_arr, eventcount_arr
#     elif channels[-1] == 25:
#         ys_arrHe = np.zeros((numRuns,numSamples[-1]), dtype=np.int64)
#     #ETTT_arr = []
#     #eventcount_arr = []
#     #print(ys_arr)
    for i in range(0,len(channels)):
        eventCount = 0
        byteCounter = 0
        #ys_arr.append([])
        #eventcount_arr.append([])
        #ys_arr.append(List.empty_list(uint16))
        #eventcount_arr.append(List.empty_list(intc))
        #print(ys_arr)
#         if channels[i] == 25:
#             print('3He channel loaded')
#             while byteCounter < fileLength[i]:
#                 if byteCounter == 0:
#                     #ETTT_arr[i]=TTT[i]
#                     ##ETTT_arr[i].append(TTT[i])
#                     #eventcount_arr[i]=(eventCounter[i])
#                     byteCounter = 20
#                 else:
#                     #ETTT_arr[i]=(read_data[i][byteCounter]+(read_data[i][byteCounter+1]<<16)+(read_data[i][byteCounter+2]<<32))
#                     #eventcount_arr[i]=(read_data[i][byteCounter+4]+(read_data[i][byteCounter+5]<<16))
#                     byteCounter += 6
#                 for j in range(0, numSamples[i]):
#                     #if j == 0:
#                         #ys_arr[i].append([])
#                     #print(byteCounter)
#                     ys_arrHe[eventCount][j]=read_data[i][byteCounter]
#                     byteCounter += 1
#                 eventCount += 1
        #else:
            #byte counter is really 2bytecounter, lol
        while byteCounter < fileLength[i]:
            if byteCounter == 0:
                ETTT_arr[i]=TTT[i]
                #ETTT_arr[i].append(TTT[i])
                eventcount_arr[i]=(eventCounter[i])
                byteCounter = 20
            else:
                ETTT_arr[i]=(read_data[i][byteCounter]+(read_data[i][byteCounter+1]<<16)+(read_data[i][byteCounter+2]<<32))
                eventcount_arr[i]=(read_data[i][byteCounter+4]+(read_data[i][byteCounter+5]<<16))
                byteCounter += 6
            for j in range(0, numSamples[i]):
                #if j == 0:
                    #ys_arr[i].append([])
                #print(byteCounter)
                ys_arr[i][eventCount][j]=read_data[i][byteCounter]
                byteCounter += 1
            eventCount += 1
    return ys_arr, ETTT_arr, eventcount_arr

#start=time.time()
#ys_arrHe = List.empty_list(uint16)
ys_arr, ETTT_arr, eventcount_arr  = test2(read_data, chan_enab, fileLength, numSamples)
end = time.time()
print(end-start)
print(len(ys_arr))
print(len(ETTT_arr))
# print('He:')
# print(len(ys_arrHe))
# print(len(ys_arrHe[0]))
#print(len(ys_arr[0]))
#print(len(ys_arr[0][0]))


C:\Users\gabeo\anaconda3\lib\site-packages\numba\core\ir_utils.py:2152: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'fileLength' of function 'test2'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "C:\Users\gabeo\AppData\Local\Temp\ipykernel_83904\1408888969.py", line 25:
@jit(nopython = True)
def test2(read_data, channels, fileLength, numSamples):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))
C:\Users\gabeo\anaconda3\lib\site-packages\numba\core\ir_utils.py:2152: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'read_data' of function 'test2'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types


3.4172468185424805
12
12


Put ADC values in arrays for each channel (one array per event, an array of events per channel) and put the miniheader information in an array

start=time.time()
ys_arr = np.array([])
ETTT_arr = []
eventcount_arr = np.array([])

for i in range(0,len(chan_enab)):
    eventCount = 0
    byteCounter = 0
    np.append(ys_arr,np.array([]))
    np.append(eventcount_arr,np.array([]))
    
    #byte counter is really 2bytecounter, lol
    while byteCounter < fileLength[i]:
        if byteCounter == 0:
            #np.append(ETTT_arr,np.array([]))
            ETTT_arr[i].append([])
            np.append(eventcount_arr,np.array([]))
            #np.append(ETTT_arr[i],TTT[i])
            ETTT_arr[i].append(TTT[i])
            np.append(ETTT_arr,np.array([]))
            eventcount_arr[i].append(eventCounter[i])
            byteCounter = 20
        else:
            ETTT_arr[i].append(read_data[i][byteCounter]+(read_data[i][byteCounter+1]<<16)+(read_data[i][byteCounter+2]<<32))
            eventcount_arr[i].append(read_data[i][byteCounter+4]+(read_data[i][byteCounter+5]<<16))
            byteCounter += 6
        for j in range(0, numSamples[i]):
            if j == 0:
                ys_arr[i].append(np.array([]))
            ys_arr[i][eventCount].append(read_data[i][byteCounter]) 
            byteCounter += 1
        eventCount += 1
#read_data.clear()
    #times_arr.append(list(map(list, zip(*headers_arr[i][1:])))[0])
    
end = time.time()
print(end-start)

Calculate the time difference between each event within a file - used to check for dropping pulses. It seems that if we make the record window 49.152 ms long, we miss every other pulse (at 20 Hz). This is not that surprising - we presumably will not need a lot of data (or any) with the full 50 ms time window.

Plot each channels' events

In [6]:
#switchs = [31, 74, 117, 160, 203, 246, 289, 332]
#%matplotlib notebook
baseL = 0
baseR = int(((preTime[0]-groupStart[0])*0.70)/chanDec[0])
intgrL = 0 # bin 1800 looks OK for integral, but start at 0...
intgrR = 8999
HeintgrL = 7000 # 3He data is a different size
HeintgrR = 16000
sums = []
ys_basesub = [] 
legend = ['NaI', '6Li', '3He']
#statesID = ['111', '101', '100', '110','101','110','111','100','111']
#switchpulses = np.arange(230,575,43)
#print(switchpulses)
#p=8
pulse = 20
#print(pulse)

#%matplotlib qt
#for i in range(0,len(chan_enab)):
for i in range(0,3):
    tempys_basesub = []
    tempsums =[]
    s=pulse
    t=s+1
    for j in range(0, len(ys_arr[i])): ##baseline subtract all pulses and take integral of all pulses
        if i==2: ## currently 3He is i=2. change if changing # of channels
            tempys_basesub.append(ys_arr[i][j] - np.mean(ys_arr[i][j][baseL:baseR]))
            tempsums.append(sum(tempys_basesub[j][HeintgrL:HeintgrR]))  ## Need to change sums array format if also doing LO/R
        else:
            tempys_basesub.append(ys_arr[i][j] - np.mean(ys_arr[i][j][baseL:baseR]))
            #print(np.mean(ys_arr[i][j][200:6000]))
            tempsums.append(sum(tempys_basesub[j][intgrL:intgrR]))  ## Need to change sums array format if also doing LO/R
    ys_basesub.append(tempys_basesub)
    sums.append(tempsums)
    #print(len(ys_basesub))
    for j in range(s, t): ## plot only interested pulses
        #ys_basesub.append(ys_arr[i][j] - np.mean(ys_arr[i][j][200:6000]))
        #print(sum(ys_basesub[i][j])) 
        plt.plot(xs[i], tempys_basesub[j] , label=legend[i] +', Integral = ') #+str(sums[1][j])) ## sums[j] will not work for more than just TR   

#plt.xlim(-50000, 900000)
#plt.ylim(3000,4100)
plt.title('Detector signals') 
plt.xlabel("time from trigger (ns)")
plt.ylabel("ADC")
print(len(xs[0]))
print(len(ys_arr[0][0]))
plt.axvline(xs[0][baseL], ls = '--')
plt.axvline(xs[0][baseR], ls = '--')
plt.axvline(xs[2][HeintgrL], ls = '--', c ='r')
plt.axvline(xs[2][HeintgrR], ls = '--', c ='r')

#plt.text(1.5e7, 1200, 'test', fontsize=12)
plt.legend()
plt.show()

print('sums length:'+ str(len(sums)))
print(len(sums[2]))

#print(np.array(expectedSumsTR)+np.array(expectedSumsR))

print(len(sums))
print(len(ys_arr[2][0]))
print(len(ys_arr[1][0]))
print(len(xs[2]))
#plt.hist(sums, 100)
#plt.show(

9000
9000
sums length:3
5000
3
45000
9000
45000


In [7]:
## Divide NaI and 6Li by 3He integral (normalize), then take integral of NaI and 6Li
baseL = 0
baseR = 1500
intgrL = 0 # bin 1800 looks OK for integral, but start at 0...
intgrR = 8999
HeintgrL = 7000 # 3He data is a different size
HeintgrR = 16000
#res =2.3 ms
sums = []
sums_norm = []
ys_basesub = []
ys_base_norm = [] 
legend = ['NaI', '6Li', '3He']
#statesID = ['111', '101', '100', '110','101','110','111','100','111']
#switchpulses = np.arange(230,575,43)
#print(switchpulses)
#p=8
pulse = 20
#print(pulse)
## t_to_eV = 0.5*(1.6749286*10**(-27))*((27.3/TIME))**2*(6.24151*10**18 [J/eV])

#%matplotlib qt
#for i in range(0,len(chan_enab)):
for i in (range(0,3)):
    tempys_basesub = []
    tempsums =[]
    s=pulse
    t=s+1
    for j in range(0, len(ys_arr[i])): ##baseline subtract all pulses and take integral of all pulses
        if i==2: ## currently 3He is i=2. change if changing # of channels
            tempys_basesub.append(ys_arr[i][j] - np.mean(ys_arr[i][j][baseL:baseR]))
            tempsums.append(sum(tempys_basesub[j][HeintgrL:HeintgrR]))  ## Need to change sums array format if also doing LO/R
        else:
            tempys_basesub.append(ys_arr[i][j] - np.mean(ys_arr[i][j][baseL:baseR]))
            #print(np.mean(ys_arr[i][j][200:6000]))
            tempsums.append(sum(tempys_basesub[j][intgrL:intgrR]))  ## Need to change sums array format if also doing LO/R
    ys_basesub.append(tempys_basesub)
    sums.append(tempsums)
    #print(len(ys_basesub))
    for j in range(s, t): ## plot only interested pulses
        #ys_basesub.append(ys_arr[i][j] - np.mean(ys_arr[i][j][200:6000]))
        #print(sum(ys_basesub[i][j])) 
        plt.plot(xs[i], tempys_basesub[j] , label=legend[i] +', Integral = ') #+str(sums[1][j])) ## sums[j] will not work for more than just TR   
 
        
## now normalize NaI and 6Li (can we do this in the forloop above?)

print(sums[2][pulse])
print((ys_basesub[1][pulse]))
print((ys_basesub[1][pulse]/sums[2][pulse]))

for i in (range(0,2)):
    tempys_base_norm = []
    tempsums =[]
    s=pulse
    t=s+1
    for j in range(0, len(ys_arr[i])): ##baseline subtract all pulses and take integral of all pulses
        tempys_base_norm.append(ys_basesub[i][j]/(sums[2][j]/1000000))
        tempsums.append(sum(tempys_base_norm[j][intgrL:intgrR]))  ## Need to change sums array format if also doing LO/R    
    ys_base_norm.append(tempys_base_norm)
    sums_norm.append(tempsums)
    #print(len(ys_basesub))
    for j in range(s, t): ## plot only interested pulses
        #ys_basesub.append(ys_arr[i][j] - np.mean(ys_arr[i][j][200:6000]))
        #print(sum(ys_basesub[i][j])) 
        plt.plot(xs[i], tempys_base_norm[j] , label=legend[i] +', normalized') #+str(sums[1][j])) ## sums[j] will not work for more than just TR   
        

#plt.xlim(-50000, 900000)
#plt.ylim(3000,4100)
plt.title('Detector signals') 
plt.xlabel("time from trigger (ns)")
plt.ylabel("ADC")
print(len(xs[0]))
print(len(ys_arr[0][0]))
plt.axvline(xs[0][baseL], ls = '--')
plt.axvline(xs[0][baseR], ls = '--')
plt.axvline(xs[1][intgrL], ls = '--', c ='g')
plt.axvline(xs[1][intgrR], ls = '--', c ='g')
plt.axvline(xs[2][HeintgrL], ls = '--', c ='r')
plt.axvline(xs[2][HeintgrR], ls = '--', c ='r')

#plt.text(1.5e7, 1200, 'test', fontsize=12)
plt.legend()
plt.show()

print('sums length:'+ str(len(sums)))
print(len(sums[2]))

#print(np.array(expectedSumsTR)+np.array(expectedSumsR))

print((sums[1][0]))
print((sums_norm[1][0]))
print((sums[1][0]/(sums[2][0]/1000000))) ##These are basically the same, so we can just use sums/normfactor
#plt.hist(sums, 100)
#plt.show(

6784184.000000795
[-0.066 -0.066 -0.066 ... -1.066 -1.066 -1.066]
[-9.72850972e-09 -9.72850972e-09 -9.72850972e-09 ... -1.57130172e-07
 -1.57130172e-07 -1.57130172e-07]
9000
9000
sums length:3
5000
-63365.88066668381
-9536.136309948082
-9536.136309950876


## Add up all 5000 pulses


In [9]:
## probably want to change this to make example for thesis

baseL = 0
baseR = 1500
intgrL = 0 # bin 1800 looks OK for integral, but start at 0...
intgrR = 8999
HeintgrL = 7000 # 3He data is a different size
HeintgrR = 16000
#res =2.3 ms
sums = []
sums_norm = []
ys_basesub = []
ys_base_norm = [] 
legend = ['NaI', '6Li', '3He']
#statesID = ['111', '101', '100', '110','101','110','111','100','111']
#switchpulses = np.arange(230,575,43)
#print(switchpulses)
#p=8
pulse = 20
#print(pulse)
## t_to_eV = 0.5*(1.6749286*10**(-27))*((27.3/TIME))**2*(6.24151*10**18 [J/eV])

#%matplotlib qt
#for i in range(0,len(chan_enab)):
for i in (range(0,3)):
    tempys_basesub = []
    tempsums =[]
    s=pulse
    t=s+1
    for j in range(0, len(ys_arr[i])): ##baseline subtract all pulses and take integral of all pulses
        if i==2: ## currently 3He is i=2. change if changing # of channels
            tempys_basesub.append(ys_arr[i][j] - np.mean(ys_arr[i][j][baseL:baseR]))
            tempsums.append(sum(tempys_basesub[j][HeintgrL:HeintgrR]))  ## Need to change sums array format if also doing LO/R
        else:
            tempys_basesub.append(ys_arr[i][j] - np.mean(ys_arr[i][j][baseL:baseR]))
            #print(np.mean(ys_arr[i][j][200:6000]))
            tempsums.append(sum(tempys_basesub[j][intgrL:intgrR]))  ## Need to change sums array format if also doing LO/R
    ys_basesub.append(tempys_basesub)
    sums.append(tempsums)
    #print(len(ys_basesub))
    #for j in range(s, t): ## plot only interested pulses
        #ys_basesub.append(ys_arr[i][j] - np.mean(ys_arr[i][j][200:6000]))
        #print(sum(ys_basesub[i][j])) 
        #plt.plot(xs[i], tempys_basesub[j] , label=legend[i] +', Integral = ') #+str(sums[1][j])) ## sums[j] will not work for more than just TR   
 
        
## now normalize NaI and 6Li (can we do this in the forloop above?)
## also add all 5000 pulses together

added_pulses = [] 

for i in (range(0,2)):
    tempys_base_norm = []
    tempsums =[]
    added_pulses.append(np.zeros(len(ys_basesub[i][0]))) ## make zeros array of length equal to window of cahnnel 
    s=pulse
    t=s+1
    for j in range(0, len(ys_arr[i])): ##baseline subtract all pulses and take integral of all pulses
        tempys_base_norm.append(ys_basesub[i][j]/(sums[2][j]/1000000))
        tempsums.append(sum(tempys_base_norm[j][intgrL:intgrR]))## Need to change sums array format if also doing LO/R    
        added_pulses[i] = added_pulses[i]+np.array(tempys_base_norm[j])
    ys_base_norm.append(tempys_base_norm)
    sums_norm.append(tempsums)
    #print(len(ys_basesub))
    #for j in range(s, t): ## plot only interested pulses
        #ys_basesub.append(ys_arr[i][j] - np.mean(ys_arr[i][j][200:6000]))
        #print(sum(ys_basesub[i][j])) 
        #plt.plot(xs[i], tempys_base_norm[j] , label=legend[i] +', normalized') #+str(sums[1][j])) ## sums[j] will not work for more than just TR         
    plt.plot(xs[i], added_pulses[i] , label=legend[i] +', 5000 pulses added') 
    
#plt.xlim(-50000, 900000)
#plt.ylim(3000,4100)
plt.title('Detector signals') 
plt.xlabel("time from trigger (ns)")
plt.ylabel("ADC")

plt.axvline(xs[0][baseL], ls = '--')
plt.axvline(xs[0][baseR], ls = '--')
#plt.axvline(xs[1][intgrL], ls = '--', c ='g')
#plt.axvline(xs[1][intgrR], ls = '--', c ='g')
#plt.axvline(xs[2][HeintgrL], ls = '--', c ='r')
#plt.axvline(xs[2][HeintgrR], ls = '--', c ='r')

#plt.text(1.5e7, 1200, 'test', fontsize=12)
plt.legend()
plt.show()

print(len(added_pulses))
print(len(added_pulses[1]))


2
9000


In [10]:
T_0 = np.genfromtxt("/N/u/goterom/Quartz/Desktop/unpol_09919.csv", delimiter=",")
T_0 = T_0[2001:-1]
T_n = added_pulses[1][2001:-1]
print(len(T_0**2/T_n**2))
x_E = 3.895662*10**(-6)*(1/(xs[1][2001:-1]*10**(-9))**2)
print((x_E[380]))
print(xs[1][2400])
print(3.895662*10**(-6)*(1/(512*10**(-9)))**2)
print((512*10**(-9)))


P_E = np.sqrt(np.absolute(1 - (T_0**2/T_n**2)))

plt.plot(x_E[380:-1], P_E[380:-1]) 
    
#plt.xlim(-50000, 900000)
#plt.ylim(3000,4100)
plt.title('3He Polarization P(E)') 
plt.xlabel("E [eV]")
plt.ylabel("P(E)")

#plt.axvline(xs[0][baseL], ls = '--')
#plt.axvline(xs[0][baseR], ls = '--')
#plt.axvline(xs[1][intgrL], ls = '--', c ='g')
#plt.axvline(xs[1][intgrR], ls = '--', c ='g')
#plt.axvline(xs[2][HeintgrL], ls = '--', c ='r')
#plt.axvline(xs[2][HeintgrR], ls = '--', c ='r')

#plt.text(1.5e7, 1200, 'test', fontsize=12)
plt.legend()
plt.show()


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


6998
102.37440620551811
204800.0
14860771.179199215
5.12e-07


Now get the final sum over all pulses

In [9]:
## sum the sums... Only 6Li is important for this, but keeping it general

legend = ['NaI', '6Li']
sumTot = []
for i in (range(0,2)):
    tempsum = []
    tempsum.append(sum(sums[i]/(np.array(sums[2])/1000000)))
    sumTot.append(tempsum)
    
print(np.mean(sums[2]))
    
print(sumTot)
print('Sum for all spin states of run' + run_num + ': ' + str(sumTot[1][0]))

6652627.0427999925
[[726399252.2521336], [-47553734.12199138]]
Sum for all spin states of run12400: -47553734.12199138


Plot in terms of eV?

In [26]:
## Need to look after 0. Only look at resonance regions.
## t_to_eV = 0.5*(1.6749286*10**(-27))*((27.3/TIME))**2*(6.24151*10**18 [J/eV])
testeV = 0.5*(1.6749286*10**(-27))*(27.3/(np.array(xs[1][2001:])*10**9))**2*(6.24151*10**18)
for i in (range(0,2)):
    tempys_basesub = []
    tempsums =[]
    s=pulse
    t=s+1
    for j in range(0, len(ys_arr[i])): ##baseline subtract all pulses and take integral of all pulses
        if i==2: ## currently 3He is i=2. change if changing # of channels
            tempys_basesub.append(ys_arr[i][j] - np.mean(ys_arr[i][j][baseL:baseR]))
            tempsums.append(sum(tempys_basesub[j][HeintgrL:HeintgrR]))  ## Need to change sums array format if also doing LO/R
        else:
            tempys_basesub.append(ys_arr[i][j] - np.mean(ys_arr[i][j][baseL:baseR]))
            #print(np.mean(ys_arr[i][j][200:6000]))
            tempsums.append(sum(tempys_basesub[j][intgrL:intgrR]))  ## Need to change sums array format if also doing LO/R
    ys_basesub.append(tempys_basesub)
    sums.append(tempsums)
    #print(len(ys_basesub))
    for j in range(s, t): ## plot only interested pulses
        #ys_basesub.append(ys_arr[i][j] - np.mean(ys_arr[i][j][200:6000]))
        #print(sum(ys_basesub[i][j])) 
        plt.plot(testeV, tempys_basesub[j][2001:] , label=legend[i] +', Integral = ') #+str(sums[1][j])) ## sums[j] will not work for more than just TR   

plt.title('Detector signals') 
plt.xlabel("eV")
plt.ylabel("ADC")
print(len(xs[0]))
print(len(ys_arr[0][0]))
#plt.axvline(xs[0][baseL], ls = '--')
#plt.axvline(xs[0][baseR], ls = '--')
#plt.axvline(testeV[intgrL], ls = '--', c ='g')
#plt.axvline(testeV[intgrR], ls = '--', c ='g')
#plt.axvline(xs[2][HeintgrL], ls = '--', c ='r')
#plt.axvline(xs[2][HeintgrR], ls = '--', c ='r')

#plt.text(1.5e7, 1200, 'test', fontsize=12)
plt.legend()
plt.show()
print(xs[1][2001:])
print(testeV)

9000
9000
[5.120000e+02 1.024000e+03 1.536000e+03 ... 3.582464e+06 3.582976e+06
 3.583488e+06]
[1.48607711e-29 3.71519278e-30 1.65119679e-30 ... 3.03541166e-37
 3.03454421e-37 3.03367714e-37]


In [8]:
sumsLO = []
sumsTR = []
sumsR = []

ys_basesub = []
legend = ['LO', 'TR', 'R']
statesID = ['111', '101', '100', '110','101','110','111','100','111']
switchpulses = np.arange(230,575,43)

for j in switchpulses:
    ys_basesub.append(ys_arr[0] - np.mean(ys_arr[0][j][baseL:baseR]))
    sumsLO.append(sum(ys_basesub[0][j][60:-1]))
    #print((len(ys_basesub)))
    ys_basesub = []
for j in switchpulses:
    ys_basesub.append(ys_arr[1] - np.mean(ys_arr[1][j][baseL:baseR]))
    sumsTR.append(sum(ys_basesub[0][j][60:-1]))
    #print((len(ys_basesub)))
    ys_basesub = []
for j in switchpulses:
    ys_basesub.append(ys_arr[2] - np.mean(ys_arr[2][j][baseL:baseR]))
    sumsR.append(sum(ys_basesub[0][j][60:-1]))
    #print((len(ys_basesub)))
    ys_basesub = []
    
sumsLO =np.array(sumsLO)
sumsTR =np.array(sumsTR)
sumsR =np.array(sumsR)
    
print(sumsLO)
print(sumsTR)
print(sumsR)
print(sumsTR+sumsR)

[974515.00199964 936179.22399994 975085.73533334 946074.0853335
 956966.17666671 940943.62866666 963640.02533324 945226.96800023
 993454.70400038]
[-63718.27533335 -62157.608      -63476.67466667 -62421.94266669
 -61866.         -61884.34933335 -63441.75600001 -64378.96666666
 -62743.83733334]
[37273269.79600207 37690246.97534942 37648681.45802363 37170611.38800599
 36896463.00266942 37285983.69603391 37598406.80201846 37789491.5986473
 37631863.84996781]
[37209551.52066872 37628089.36734942 37585204.78335696 37108189.44533931
 36834597.00266942 37224099.34670056 37534965.04601846 37725112.63198064
 37569120.01263448]


In [9]:
##Threshold testing
TRThresh = 32 ## Note the threshold can change from statechange to statechange. Need to figure out a good value.
RThresh = 30
ys_basesub = []
where = []
pulseswitch_list = []
zeros=0
zeros_arr = []
switchbin = []
States_full = [zeros_arr,switchbin]

#for i in range(0,len(chan_enab)):
for i in range(1,2):
    s=pulse
    t=s+1
    for j in range(0, len(ys_arr[i])):
    #for j in range(s, t):
        ys_basesub.append(ys_arr[i][j] - np.mean(ys_arr[i][j][200:6000]))
        thresh_xing = np.diff(np.absolute(ys_basesub[j]) > TRThresh, prepend=False)
        xing = np.argwhere(thresh_xing)[:,0]
        #print(len(xing))
        if len(xing)== 0:
            #pulseswitch_list.append(0)  ## can now get rid of this, but illustrrates non-switch and switch pulses
            zeros+=1
        else:
            #pulseswitch_list.append(xing[0])  ## can now get rid of this, but illustrrates non-switch and switch pulses
            switchbin.append(j)
            zeros_arr.append(zeros)
            zeros=0
        
#for pulse in pulseswitch_list:
#    #print(pulse)
#    if pulse==0:
#        #print(zeros[0])
#        zeros+=1
#    else:
#        switchbin.append(pulse)
#        zeros_arr.append(zeros)
#        zeros=0
        
States_full = np.array(States_full).T
            
for state in States_full[1:]:
    if state[0]!=42:
        print('SF ERROR!')
    else:
        print('SF OK')
        #where.append(np.where(ys_basesub[j] > TRThresh))
        #print(np.mean(ys_arr[i][j][200:6000]))
        #print(sum(ys_basesub[i][j]))
        #sums.append(sum(ys_basesub[i][j][9000:-1]))
        #plt.plot(xs[i], ys_basesub[i][j], label=legend[i]+', Integral = '+str(sums[i]))

#print(pulseswitch_list)
print(States_full)
#print(zeros)

SF OK
SF OK
SF OK
SF OK
SF OK
SF ERROR!
SF OK
SF OK
SF OK
SF OK
SF OK
SF OK
SF ERROR!
SF OK
SF OK
SF OK
SF OK
SF OK
SF OK
SF ERROR!
SF OK
SF OK
SF OK
SF OK
SF OK
SF OK
SF ERROR!
SF OK
SF OK
SF OK
SF OK
SF OK
SF OK
SF ERROR!
SF OK
SF OK
SF OK
SF OK
SF OK
SF OK
SF ERROR!
SF OK
SF OK
SF OK
SF OK
SF OK
SF OK
SF ERROR!
SF OK
SF OK
SF OK
SF OK
SF OK
SF OK
SF ERROR!
SF OK
SF OK
SF OK
SF OK
SF OK
SF OK
SF ERROR!
SF OK
SF OK
SF OK
SF OK
SF OK
SF OK
SF ERROR!
SF OK
SF OK
SF OK
SF OK
SF OK
SF OK
SF ERROR!
SF OK
SF OK
SF OK
SF OK
SF OK
[[  15   15]
 [  42   58]
 [  42  101]
 [  42  144]
 [  42  187]
 [  42  230]
 [  85  316]
 [  42  359]
 [  42  402]
 [  42  445]
 [  42  488]
 [  42  531]
 [  42  574]
 [  85  660]
 [  42  703]
 [  42  746]
 [  42  789]
 [  42  832]
 [  42  875]
 [  42  918]
 [  85 1004]
 [  42 1047]
 [  42 1090]
 [  42 1133]
 [  42 1176]
 [  42 1219]
 [  42 1262]
 [  85 1348]
 [  42 1391]
 [  42 1434]
 [  42 1477]
 [  42 1520]
 [  42 1563]
 [  42 1606]
 [  85 1692]
 [  42 1735]
 [

BELOW IS TESTING

In [9]:
#print(np.mean(ys_arr[0][0:6000]))
ys_basesub = []

#for i in range(0,len(chan_enab)):
#    ys_basesub.append(ys_arr[i] - np.mean(ys_arr[i][0:6000]))
#    print(np.mean(ys_arr[i][0:6000]))

for i in range(0,len(chan_enab)):
#for i in range(0,1):
    s=pulse
    t=s+1
    #for j in range(0, len(ys_arr[i])):
    for j in range(s, t):
        ys_basesub.append(ys_arr[i] - np.mean(ys_arr[i][j][300:6000]))
        print(np.mean(ys_arr[i][j][300:6000]))
        sums.append(sum(ys_arr[i][j]))
        plt.plot(xs[i], ys_basesub[i][j], label=legend[i]+', Integral = '+str(sums[i]))

#plt.xlim(-50000, 900000)
#plt.ylim(3000,4100)
plt.title('SF state transition '+statesID[p]+' -> '+statesID[p+1])
plt.xlabel("time from trigger (ns)")
plt.ylabel("ADC")
#plt.text(1.5e7, 1200, 'test', fontsize=12)
plt.legend()
plt.show()
    
print(len(ys_basesub[0][26][0:6000]))

1062.9626315789474
1080.0126315789473
1092.0129824561404
6000


In [10]:
#switchs = [31, 74, 117, 160, 203, 246, 289, 332]
#%matplotlib notebook
sums = []
switchs = np.arange(31,1923,43)
print(switchs)
ys_basesub = []

for i in range(0,len(chan_enab)):
    ys_basesub.append(ys_arr[i] - np.mean(ys_arr[0][0:6000]))
    
#%matplotlib qt
#for i in range(0,len(chan_enab)):
#for i in range(2,5):
#    s=375
#    t=s+1
#    #for j in range(0, len(ys_arr[i])):
#    for j in range(s, t):
#        plt.plot(xs[i], ys_arr[i][j])
#for i in range(2,5):
#    s=332
#    t=s+1
#    #for j in range(0, len(ys_arr[i])):
#    for j in range(s, t):
#        plt.plot(xs[i], ys_arr[i][j])
for i in range(4,5):
    for j in switchs:
        plt.plot(xs[i], ys_arr[i][j])
        sums.append(sum(ys_arr[i][j]))
#plt.xlim(-50000, 900000)
#plt.ylim(3000,4100)
plt.xlabel("time from trigger (ns)")
plt.show()
print(sums)
print(len(ys_arr[1]))
plt.hist(sums, 100)
plt.show()

[  31   74  117  160  203  246  289  332  375  418  461  504  547  590
  633  676  719  762  805  848  891  934  977 1020 1063 1106 1149 1192
 1235 1278 1321 1364 1407 1450 1493 1536 1579 1622 1665 1708 1751 1794
 1837 1880]


IndexError: list index out of range

In [ ]:
pulse = 289
for i in range(4,5):
    for j in switchs:
        plt.plot(xs[i], ys_arr[i][pulse])
print(ys_arr[4][pulse])
print(sum(ys_arr[4][pulse]))

summation for all channels

In [ ]:
NaIthres=3500
Li6thres=200
Hethres=0
NaIsign="pos"
Li6sign="neg"
Hesign=""
threshold_array=[NaIthres,NaIthres,NaIthres,NaIthres,NaIthres,NaIthres,NaIthres,NaIthres,NaIthres,NaIthres,
                 NaIthres,NaIthres,NaIthres,NaIthres,NaIthres,NaIthres,NaIthres,NaIthres,NaIthres,NaIthres,
                NaIthres,NaIthres,NaIthres,NaIthres,Li6thres,Hethres]
threshold_sign=[NaIsign,NaIsign,NaIsign,NaIsign,NaIsign,NaIsign,NaIsign,NaIsign,NaIsign,NaIsign,NaIsign,
                NaIsign,NaIsign,NaIsign,NaIsign,NaIsign,NaIsign,NaIsign,NaIsign,NaIsign,NaIsign,NaIsign,
                NaIsign,NaIsign,Li6sign,Hesign]
xCrosses=[]#outer array is crossing arrays for given channel, inner array is crossing for each event
modeCrosses=[] #single array with one value per channel
xValues=[]
xtimems=[]

extendedRange=3
for i in range(0,len(chan_enab)):
    xCrosses.append([])
    xValues.append([])
    for j in ys_arr[i]:
        newj = np.array(j)
        if threshold_sign[i] == "pos":
            thresh_xing = np.diff(newj > threshold_array[i], prepend=False)
        elif threshold_sign[i] == "neg":
            thresh_xing = np.diff(newj < threshold_array[i], prepend=False)
        xing = np.argwhere(thresh_xing)[:,0]
        xCrosses[i].append(xing[0])
    modeCrosses.append(st.mode(xCrosses[i])) #find the most typical crossing value for each channel
    xValues[i].append(np.arange(len(ys_arr[i][0])+2*extendedRange)-modeCrosses[i])
    xtimems.append(xValues[i][-1]*resolution[i]*10**(-6))
print(len(xtimems[1]))
print(modeCrosses)
print(xValues[0])

                       

In [ ]:
#print(xCrosses)
extendedRange = 3


time_array=[]
time_nparray=[]
failedRuns=[]
for i in range(0,len(chan_enab)):
#for i in range(0,2):
    time_array.append(np.zeros(len(ys_arr[i][0])+2*extendedRange))
    time_nparray.append([])
    for j in range(0, len(ys_arr[i])):
        shiftedWave = ys_arr[i][j][:] - np.mean(ys_arr[i][j][0:int(xing[0]*0.80)]) #this subtracts the baseline -
        #baseline is based on the first 80% of samples before the threshold crossing
        offset = xCrosses[i][j]-modeCrosses[i]
        if (abs(offset) < extendedRange+1):
            for k in range(len(shiftedWave)):
                time_array[i][k-offset+extendedRange]=time_array[i][k-offset+extendedRange]+shiftedWave[k]
                #may need to switch this to subtracting offset
        else:
            failedRuns.append([i,j])
    time_nparray[i]=np.array(time_array[i])
#print(xCrosses)
print(len(time_nparray[0]))

In [ ]:
for i in range(0,len(chan_enab)):
#for i in range(0,1):
    plt.plot(xtimems[i],time_nparray[i])
    plt.xlabel('Time [ms]')
    plt.ylabel('counts (arb)')
    #plt.xlim(-0.1,0.2)
    #plt.ylim(-600000,0)
plt.show()

In [ ]:

d = {"right": [0,1,8,9,10,11,14,15,16,17,18,19],
     "left": [2,3,4,5,6,7,12,13,20,21,22,23],
     "up": [12,13,14,15,16,17,18,19,20,21,22,23],
     "down": [0,1,2,3,4,5,6,7,8,9,10,11],
     "corner": [0,3,6,9,12,15,18,21],
     "noncorner": [1,2,4,5,7,8,10,11,13,14,16,17,19,20,22,23],
     "top": [0,1,2,3,4,11,12,13,14,15,16,23],
     "bottom": [5,6,7,8,9,10,17,18,19,20,21,22],
     "Li6": [24],
     "NaI": [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23],
     "He3": [25]
    }

In [ ]:
type1="up"
type2="down"
type1_array=np.zeros(len(ys_arr[0][0]))
type2_array=np.zeros(len(ys_arr[0][0]))
for i in d[type1]: 
    type1_array = np.add(type1_array, time_nparray[i])
    
for i in d[type2]:
    type2_array = np.add(type2_array, time_nparray[i])
    plt.title(type1 + " in blue, " + type2 + " in red")
plt.plot(xtimems[0], type1_array/len(d[type1]),'b', lw=1)
plt.plot(xtimems[0], type2_array/len(d[type2]),'r',lw=1)
plt.show()